In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import warnings
import re

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

## Dataframe Creation

### Initial

In [3]:
def check_id(old_id):
    url = f'https://www.wowhead.com/item={old_id}?xml'
    html = requests.get(url).text
    soup = BeautifulSoup(html, features='xml')
    name = soup.find('name').text
    text = soup.find('htmlTooltip').text
    
    #check for quality tier information
    if text.find('quality-tier1') >= 0:
        new_id = old_id+2
    elif text.find('quality-tier2') >= 0:
        new_id = old_id+1
    else: #either its tier3 or it doesn't have tiers, in which use the old_id
        return old_id
    
    
    #wasn't a tier 3 item, so check the calculated id for if the name matches and is tier 3
    #return the new id if it is the same name and tier 3, else return -1 for manual checking
    url = f'https://www.wowhead.com/item={new_id}?xml'
    html = requests.get(url).text
    soup = BeautifulSoup(html, features='xml')
    
    try:
        if  soup.find('name').text == name and soup.find('htmlTooltip').text.find('quality-tier3') >= 0:
            return new_id
    except:
        return -1
    
    return -1

In [4]:
class Profession:
    __all_data = None
    __name = None
    
    def __init__(self, profession):
        self.__all_data = list()
        self.__name = profession
        
    def add(self, item, item_id, reagents, learned, tag, difficulty, insp_percent, multicraft, quantity, skill,
            rarity, has_reagent_qualities, has_embellishment_slot, has_missive_slot, has_tinker_slot, has_crest_slot):

            self.__all_data.append([self.__name, learned, item_id, item, None, reagents, tag, rarity, difficulty, 
                                    skill, difficulty/20, difficulty/2, insp_percent, quantity, multicraft, 
                                    has_reagent_qualities, has_embellishment_slot, has_missive_slot, has_tinker_slot, 
                                    has_crest_slot])
            
    def get_table(self):
        columns = ['profession', 'character', 'id', 'item', 'icon', 'reagents', 'tag', 'rarity', 
                   'difficulty', 'skill1', 'hsv', 'inspiration_skill', 'inspiration_percent', 
                   'base_quantity', 'multicraft_percent', 'has_reagent_qualities', 'has_embellishment_slot', 
                   'has_missive_slot', 'has_tinker_slot', 'has_crest_slot']
        dtypes = ['string', 'string', 'int32', 'string', 'string', 'object', 'string', 'string', float, float, float, 
                  float, float, 'string', float, bool, bool, bool, bool, bool]
        df = pd.DataFrame(columns=columns, data=self.__all_data)
        return df.astype(dict(zip(columns, dtypes)))
        
    def set_table(self, df):
        self.__all_data = df.to_numpy()

In [5]:
items_columns = ['id', 'item', 'icon', 'tag', 'rarity']
items_dtypes = ['int32', 'string', 'string', 'string', 'string']
items = pd.DataFrame(columns=items_columns)

professions_columns = ['profession', 'id', 'reagents', 'has_reagent_qualities', 'has_embellishment_slot',
                       'has_missive_slot', 'has_tinker_slot', 'has_crest_slot']
professions_dtypes = ['string', 'int32', dict, bool, bool, bool, bool, bool]
professions = pd.DataFrame(columns=professions_columns)

crafting_columns = ['id', 'difficulty', 'character', 'skill1', 'hsv', 'inspiration_skill', 'inspiration_percent', 
                    'base_quantity', 'multicraft_percent']
crafting_dtypes = ['int32', 'int16', 'string', float, float, float, float, 'string', float]
crafting = pd.DataFrame(columns=crafting_columns)

### Alchemy

In [6]:
# Last Updates:
# Trees: 155/155 && 155/155 && 200/200
# Gear: 53 Insp + 35 Multi + 6 Skill && 35 Res + 53 Speed + 6 Skill && (110 Multi + 45 Res || 110 Insp + 40 Insp) + 10 Skill 

alchemy = Profession('Alchemy')

### Reagents ###
alchemy.add("Primal Convergent", 191495, {190316: 1, 190321: 1, 190327: 1, 190329: 1, 190324: 1}, "Trillithia", 'Crafting Reagent', difficulty=275, insp_percent=34.8, multicraft=8.2, quantity=2, skill=222, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Omnium Draconis", 191498, {191472: 1, 191466: 1, 191462: 1, 191469: 1}, "Trillithia", 'Crafting Reagent', difficulty=325, insp_percent=34.8, multicraft=8.2, quantity=1, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Air Potions ###
alchemy.add("Residual Neural Channeling Agent", 191374, {190327: 1, 190316: 1, 191476: 1, 191466: 1}, "Trillithia", 'Consumable', difficulty=400, insp_percent=35.8, multicraft=15.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Stinky Bright Potion", 204370, {190326: 1, 190330: 1, 204460: 50, 191476: 5, 191466: 2}, "Trillithia", 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=25.9, quantity=5, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Bottled Putrescence", 191362, {190327: 1, 190331: 1, 191476: 5, 191462: 30}, "Trillithia", 'Consumable', difficulty=450, insp_percent=36.8, multicraft=15.9, quantity=5, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of Gusts", 191398, {190327: 1, 191476: 5, 191466: 5, 191462: 20}, "Trillithia", 'Consumable', difficulty=150, insp_percent=20.8, multicraft=25.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of Shocking Disclosure", 191401, {190327: 1, 190316: 1, 191476: 5, 191462: 25}, "Trillithia", 'Consumable', difficulty=150, insp_percent=20.8, multicraft=25.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of the Hushed Zephyr", 191395, {190327: 1, 191476: 5, 191462: 20, 191466: 8}, "Trillithia", 'Consumable', difficulty=150, insp_percent=20.8, multicraft=25.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Aerated Mana Potion", 191386, {190326: 3, 191476: 5, 191462: 8}, "Trillithia", 'Consumable', difficulty=60, insp_percent=20.8, multicraft=25.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Cauldrons ###
alchemy.add("Draconic Phial Cauldron", 204680, {204463: 2, 191570: 60, 190329: 5, 190327: 5, 191462: 240, 191472: 80, 191466: 80, 191469: 80}, "Trillithia", 'Consumable', difficulty=410, insp_percent=38.8, multicraft=15.9, quantity=1, skill=312, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Cauldron of Extracted Putrescence", 204633, {190331: 3, 204460: 200, 204463: 1, 204634: 1}, "Trillithia", 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion Cauldron of Ultimate Power", 191487, {191389: 150, 191498: 20}, "Trillithia", 'Consumable', difficulty=450, insp_percent=36.8, multicraft=15.9, quantity=1, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion Cauldron of Power", 191484, {191389: 100, 191498: 10}, "Trillithia", 'Consumable', difficulty=425, insp_percent=36.8, multicraft=15.9, quantity=1, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Cauldron the Pooka", 191490, {201405: 1, 191498: 1, 191495: 2}, "Trillithia", 'Consumable', difficulty=300, insp_percent=21.8, multicraft=25.9, quantity=2, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Frost Potions ###
alchemy.add("Delicate Suspension of Spores", 191377, {190329: 1, 190331: 1, 191476: 5, 191469: 10}, "Trillithia", 'Consumable', difficulty=450, insp_percent=36.8, multicraft=15.9, quantity=5, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Dreamwalker's Healing Potion", 207023, {190328: 1, 191476: 5, 191380: 5, 191365: 5}, 'Trillithia', 'Consumable', difficulty=325, insp_percent=35.8, multicraft=15.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of Chilled Clarity", 191368, {190329: 1, 190331: 1, 191476: 5, 191469: 10}, "Trillithia", 'Consumable', difficulty=450, insp_percent=36.8, multicraft=15.9, quantity=5, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of Frozen Focus", 191365, {190329: 1, 191476: 5, 191462: 20, 191466: 8}, "Trillithia", 'Consumable', difficulty=400, insp_percent=35.8, multicraft=15.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of Withering Dreams", 207041, {190328: 1, 190330:1, 191476:5, 191371: 5}, 'Trillithia', 'Consumable', difficulty=415, insp_percent=36.8, multicraft=15.9, quantity=5, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of Withering Vitality", 191371, {190329: 1, 190331: 1, 191476: 5, 191472: 10}, "Trillithia", 'Consumable', difficulty=450, insp_percent=36.8, multicraft=15.9, quantity=5, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Potion of Frozen Fatality", 191353, {190329: 1, 191476: 5, 191472: 5, 191462: 20}, "Trillithia", 'Consumable', difficulty=200, insp_percent=20.8, multicraft=25.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Refreshing Healing Potion", 191380, {190328: 1, 191476: 5, 191462: 8}, "Trillithia", 'Consumable', difficulty=40, insp_percent=20.8, multicraft=25.9, quantity=5, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Elemental Phials and Potions ###
alchemy.add("Elemental Potion of Ultimate Power", 191383, {190324: 2, 191389: 30}, "Trillithia", 'Consumable', difficulty=450, insp_percent=36.8, multicraft=15.9, quantity=20, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Elemental Potion of Power", 191389, {191476: 5, 191498: 3, 191495: 1}, "Trillithia", 'Consumable', difficulty=425, insp_percent=36.8, multicraft=15.9, quantity=5, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Phial of Elemental Chaos", 191359, {191476: 2, 191495: 1, 191498: 2}, "Trillithia", 'Consumable', difficulty=450, insp_percent=38.8, multicraft=15.9, quantity=2, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Air Phials ###
alchemy.add("Phial of Charged Isolation", 191332, {190327: 1, 190316: 1, 191476: 2, 191466: 9}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Phial of Static Empowerment", 191338, {190327: 1, 190316: 1, 191476: 2, 191469: 5}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Phial of Still Air", 191323, {190327: 1, 191476: 2, 191469: 10, 191466: 5}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Phial of the Eye in the Storm", 191320, {190327: 1, 191476: 2, 191469: 5, 191466: 10}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Aerated Phial of Deftness", 191344, {190456: 10, 190327: 2, 191476: 10, 191466: 10}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=10, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Charged Phial of Alacraity", 191350, {190327: 1, 190316: 1, 191476: 2, 191472: 4}, "Trillithia", 'Consumable', difficulty=200, insp_percent=21.8, multicraft=25.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Aerated Phial of Quick Hands", 197722, {190326: 4, 191476: 2, 191462: 8}, "Trillithia", 'Consumable', difficulty=200, insp_percent=21.8, multicraft=25.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Frost Phials ###
alchemy.add("Iced Phial of Corrupting Rage", 191329, {190331: 1, 190329: 1, 191476: 2, 191472: 9}, "Trillithia", 'Consumable', difficulty=450, insp_percent=37.8, multicraft=15.9, quantity=2, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Phial of Icy Preservation", 191326, {190331: 1, 190329: 1, 191476: 2, 191466: 9}, "Trillithia", 'Consumable', difficulty=450, insp_percent=37.8, multicraft=15.9, quantity=2, skill=312, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Phial of Glacial Fury", 191335, {190329: 1, 191476: 2, 191472: 5, 191469: 10}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Crystalline Phial of Perception", 191356, {190456: 10, 190329: 2, 191476: 10, 191469: 10}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=10, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Steaming Phial of Finesse", 191347, {190456: 10, 190329: 1, 190321: 1, 191476: 10, 191472: 8}, "Trillithia", 'Consumable', difficulty=400, insp_percent=36.8, multicraft=15.9, quantity=10, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Phial of Tepid Versatility", 191341, {190329: 2, 191476: 2, 191462: 16}, "Trillithia", 'Consumable', difficulty=300, insp_percent=36.8, multicraft=15.9, quantity=2, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Transmutations ###
alchemy.add("Transmute: Dracothyst", 204463, {204460: 300, 193370: 50}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Transmute: Decay to Elements", 192182, {190331: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Transmute: Order to Elements", 192181, {190324: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Transmute: Awakened Air", 190327, {190329: 1, 190321: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Transmute: Awakened Earth", 190316, {190329: 1, 190321: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Transmute: Awakened Fire", 190321, {190316: 1, 190327: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Transmute: Awakened Frost", 190329, {190316: 1, 190327: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Finishing Reagents ###
alchemy.add("Illustrious Insight", 191529, {190456: 50}, "Trillithia", 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Writhefire Oil", 191519, {190321: 1, 191472: 1, 191462: 2}, "Trillithia", 'Crafting Reagent', difficulty=60, insp_percent=19.8, multicraft=18.6, quantity=2, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Brood Salt", 191516, {190329: 1, 191466: 6, 191462: 12}, "Trillithia", 'Crafting Reagent', difficulty=300, insp_percent=34.8, multicraft=8.6, quantity=2, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Stable Fluidic Draconium", 191513, {190327: 1, 189143: 3, 191472: 3}, "Trillithia", 'Crafting Reagent', difficulty=300, insp_percent=34.8, multicraft=8.6, quantity=2, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Agitating Potion Augmentation", 191522, {191466: 5, 191462: 12, 191469: 6}, "Trillithia", 'Crafting Reagent', difficulty=300, insp_percent=34.8, multicraft=8.6, quantity=2, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Reactive Phial Embellishment", 191525, {191472: 6, 191462: 12, 191469: 6}, "Trillithia", 'Crafting Reagent', difficulty=300, insp_percent=34.8, multicraft=8.6, quantity=2, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Optional Reagents ###
alchemy.add("Potion Absorption Inhibitor", 191534, {191466: 10, 191462: 20, 191469: 10, 191495: 4}, 'Trillithia', 'Embellishment', difficulty=325, insp_percent=34.8, multicraft=8.6, quantity=1, skill=222, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Incense ###
alchemy.add("Sagacious Incense", 191501, {191495: 1, 191466: 5}, "Trillithia", 'Consumable', difficulty=300, insp_percent=34.8, multicraft=8.6, quantity=2, skill=222, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Exultant Incense", 191507, {191466: 6, 191462: 6}, "Trillithia", 'Consumable', difficulty=250, insp_percent=34.8, multicraft=8.6, quantity=2, skill=222, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Fervid Incense", 191510, {191472: 6, 191462: 6}, "Trillithia", 'Consumable', difficulty=150, insp_percent=19.8, multicraft=18.6, quantity=2, skill=222, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
alchemy.add("Somniferous Incense", 191504, {191469: 6, 191462: 6}, "Trillithia", 'Consumable', difficulty=250, insp_percent=34.8, multicraft=8.6, quantity=2, skill=222, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Alchemist Stones ###
alchemy.add("Alacritous Alchemist Stone", 191492, {206959: 1, 201406: 1, 191498: 15, 191389: 12, 191365: 12}, "Trillithia", 'Gear (Combat)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=232, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
alchemy.add("Sustaining Alchemist Stone", 191491, {206959: 1, 201406: 1, 191495: 4, 191341: 6, 191359: 6}, "Trillithia", 'Gear (Combat)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=232, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)

### Artisan Curios ###
alchemy.add("Draconic Suppression Powder", 203407, {203398: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ###


### Blacksmithing

In [7]:
### Aenestria ###
blacksmithing1 = Profession('Blacksmithing')

blacksmithing1.add('Illustrious Insight', 191529, {190456:50}, 'Aenestria', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Frostfire Alloy', 190532, {190321:1, 190329:1, 190452:4, 189143:5, 190314:4}, 'Aenestria', 'Crafting Reagent', difficulty=325, insp_percent=33.8, multicraft=6.5, quantity=2, skill=216, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Primal Molten Alloy', 189543, {190316:1, 190321:1, 190452:4, 189143:5, 190314:4}, 'Aenestria', 'Crafting Reagent', difficulty=325, insp_percent=33.8, multicraft=6.5, quantity=2, skill=216, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Armor Spikes', 191873, {190321:3, 201400:2, 190395:30, 190314:15}, 'Aenestria', 'Embellishment', difficulty=325, insp_percent=33.8, multicraft=6.5, quantity=1, skill=216, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Primal Razorstone', 191950, {192885:4, 193370:1}, 'Aenestria', 'Consumable', difficulty=325, insp_percent=34.8, multicraft=8.4, quantity=5, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Primal Whetstone', 191940, {190321:1, 192885:4}, 'Aenestria', 'Consumable', difficulty=325, insp_percent=34.8, multicraft=8.4, quantity=5, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Primal Weightstone', 191945, {190316:1, 192885:4}, 'Aenestria', 'Consumable', difficulty=325, insp_percent=34.8, multicraft=8.4, quantity=5, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Explorer's Expert Helm", 201951, {190452:2, 189143:7, 190395:14}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Explorer's Expert Spaulders", 201952, {190452:2, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Explorer's Expert Gauntlets", 201953, {190452:2, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Explorer's Expert Greaves", 201954, {190452:2, 189143:7, 190395:14}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Explorer's Expert Clasp", 201955, {190452:2, 189143:4, 190395:10}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Explorer's Plate Chestguard", 189538, {190452:2, 189143:3, 190395:10}, 'Aenestria', 'Gear (Combat)', difficulty=40, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Explorer's Plate Boots", 189537, {190452:2, 189143:3, 190395:10}, 'Aenestria', 'Gear (Combat)', difficulty=40, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Explorer's Plate Bracers", 189539, {190452:2, 189143:3, 190395:8}, 'Aenestria', 'Gear (Combat)', difficulty=40, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draconium Defender', 201322, {190452:2, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Crusher', 190516, {206959:2, 190535:7, 189543:7}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Hexsword', 190511, {206959:1, 190535:6, 189543:6}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Slicer', 190518, {206959:1, 190535:5, 189543:8}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Greataxe', 190510, {206959:2, 189543:20}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Longsword', 190507, {206959:1, 189543:17}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Mace', 190509, {206959:1, 189543:17}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Shortblade', 190505, {206959:1, 189543:17}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Spellblade', 190506, {206959:1, 189543:17}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Warglaive', 190508, {206959:1, 189543:17}, 'Aenestria', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Shadowed Razing Annihilator', 205046, {206959:2, 205413:2, 204995:5, 190535:7, 189543:7}, 'Aenestria', 'Gear (Combat)', difficulty=300, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draconium Great Mace', 190486, {190452:6, 189143:10, 190395:20}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draocnium Stiletto', 190482, {190452:6, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draconium Great Axe', 201324, {190452:6, 189143:10, 190395:20}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draconium Knuckles', 190485, {190452:6, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draconium Sword', 190484, {190452:5, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draconium Axe', 190487, {190452:5, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Draconium Dirk', 190483, {190452:6, 189143:6, 190395:12}, 'Aenestria', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Khaz'gorite Blacksmith's Hammer", 191888, {190456:225, 190452:15, 190314:40, 190395:100}, 'Aenestria', 'Gear (Profession)', difficulty=425, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Khaz'gorite Blacksmith's Toolbox", 191230, {190456:225, 190452:12, 190314:40, 190395:100}, 'Aenestria', 'Gear (Profession)', difficulty=400, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Khaz'gorite Pickaxe", 191223, {190456:300, 190452:15, 190314:45, 190395:100}, 'Aenestria', 'Gear (Profession)', difficulty=425, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Draconium Needle Set', 191239, {190452:3, 189143:4, 190395:10}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Draconium Leatherworker's Toolset", 191236, {190452:3, 190395:12, 189143:3}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Draconium Leatherworker's Knife", 191238, {190452:3, 189143:3, 190395:12}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Draconium Blacksmith's Toolbox", 191235, {190452:3, 190395:12, 189143:3}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Draconium Skinning Knife", 191240, {190452:4, 189143:3, 190395:10}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Draconium Sickle', 191241, {190452:4, 189143:2, 190395:10}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Draconium Pickaxe', 191242, {190452:4, 189143:2, 190395:10}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Draconium Blacksmith's Hammer", 191237, {190452:4, 189143:2, 190395:10}, 'Aenestria', 'Gear (Profession)', difficulty=80, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Master's Hammer", 201366, {190452:10, 190535:1, 189543:2, 190532:2}, 'Aenestria', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Serevite Repair Hammer', 191260, {190452:3, 190395:8}, 'Aenestria', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=12.9, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Serevite Skeleton Key', 191256, {190452:3, 190395:20}, 'Aenestria', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=12.9, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Obsidian Seared Alloy', 190535, {190324:1, 190321:1, 190452:6, 189143:10, 190314:8}, None, 'Crafting Reagent', difficulty=325, insp_percent=33.8, multicraft=6.5, quantity=2, skill=216, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Shadowed Alloy', 204994, {190321:1, 190324:1, 204464:1, 189143:5, 190314:2}, None, 'Crafting Reagent', difficulty=325, insp_percent=33.8, multicraft=6.5, quantity=2, skill=216, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Infurious Alloy', 190538, {190450:2, 190452:3, 189143:4, 190314:2}, None, 'Crafting Reagent', difficulty=200, insp_percent=18.8, multicraft=16.5, quantity=2, skill=216, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Shadowed Belt Clasp', 205039, {204995:5, 189543:13}, None, 'Item Enhancement', difficulty=340, insp_percent=31.8, multicraft=2.9, quantity=1, skill=162, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Allied Chestplate of Generosity', 190519, {206959:1, 193360:1, 190535:10}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Allied Wristguard of Companionship', 190526, {206959:1, 193360:1, 190535:8}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Frostfire Legguards of Preparation', 190523, {206959:1, 193919:1, 193362:1, 190532:16}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Infurious Helm of Vengeance', 190522, {206959:1, 190535:2, 190538:20}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Infurious Warboots of Impunity', 191985, {206959:1, 201402:2, 190535:2, 190538:16}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Breastplate', 190495, {206959:1, 189543:16}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Gauntlets', 190497, {206959:1, 189543:14}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Greatbelt', 190501, {206959:1, 189543:13}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Helm', 190498, {206959:1, 189543:16}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Legplates', 190499, {206959:1, 189543:16}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Pauldrons', 190500, {206959:1, 189543:15}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Sabatons', 190496, {206959:1, 189543:14}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Primal Molten Vambraces', 190502, {206959:1, 189543:14}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Unstable Frostfire Belt', 191623, {206959:1, 193919:1, 193362:1, 190532:13}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add("Obsidian Combatant's Draconium Armguards", 204128, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Obsidian Combatant's Draconium Breastplate", 204121, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Obsidian Combatant's Draconium Gauntlets", 204123, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Obsidian Combatant's Draconium Greaves", 204125, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Obsidian Combatant's Draconium Helm", 204124, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Obsidian Combatant's Draconium Pauldrons", 204126, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Obsidian Combatant's Draconium Sabatons", 204122, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Obsidian Combatant's Draconium Waistguard", 204127, {190538:4}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Armguards", 193609, {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Breastplate", 193602, {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Gauntlets", 193604, {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Greaves", 193606,  {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Helm", 193605, {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Pauldrons", 193607, {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Sabatons", 193603, {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Crimson Combatant's Draconium Waistguard", 193608, {190538:2}, None, 'Gear (Combat)', difficulty=120, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Alvin the Anvil', 191886, {193920:1, 189543:6, 190532:6, 190538:6}, None, 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Primal Molten Defender', 190503, {206959:1, 189543:15}, None, 'Gear (Combat)', difficulty=280, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Shadowed Impact Buckler', 205168, {206959:1, 205413:1, 204995:5, 189543:15}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Shield of the Hearth', 192081, {206959:1, 193920:1, 201406:1, 189543:16}, None, 'Gear (Combat)', difficulty=325, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=162, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Claymore', 190514, {206959:2, 190535:8, 189543:5}, None, 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Facesmasher', 190513, {206959:1, 190535:6, 189543:6}, None, 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Halberd', 190515, {206959:2, 190535:6, 189543:8}, None, 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Invoker', 190517, {206959:2, 190535:7, 189543:5}, None, 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Obsidian Seared Runeaxe', 190512, {206959:1, 190535:6, 189543:6}, None, 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing1.add('Black Dragon Touched Hammer', 191228, {190456:400, 193920:1, 190535:7}, None, 'Gear (Profession)', difficulty=450, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Khaz'gorite Leatherworker's Knife", 191227, {190456:300, 190452:15, 190314:40, 190395:100}, None, 'Gear (Profession)', difficulty=425, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Khaz'gorite Leatherworker's Toolset", 191229, {190456:300, 190452:12, 190314:40, 190395:100}, None, 'Gear (Profession)', difficulty=400, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Khaz'gorite Needle Set", 191226, {190456:300, 190452:12, 190314:35, 190395:100}, None, 'Gear (Profession)', difficulty=400, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Khaz'gorite Sickle", 191224, {190456:300, 190452:15, 190314:40, 190395:100}, None, 'Gear (Profession)', difficulty=425, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Khaz'gorite Skinning Knife", 191225, {190456:300, 190452:15, 190314:40, 190395:100}, None, 'Gear (Profession)', difficulty=425, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Heat-Resistant Rescue Ring', 205038, {205413:1, 189543:5, 190532:5, 190538:5, 204995:5}, None, 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Sturdy Expedition Shovel', 191304, {190452:2, 190395:10}, None, 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Prototype Explorer's Barding Framework", 194542, {190452:5, 201399:3, 190395:20, 189543:1, 190535:2}, None, 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add("Prototype Regal Barding Framework", 194541, {190452:5, 201403:3, 190395:20, 190532:1, 190535:2}, None, 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing1.add('Ancient Ceremonial Trident', 203408, {203399:1}, None, 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=12.9, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

In [8]:
# Last Updates:
# Trees: 390/390 && 210/210 && 110/110 && 90/90
# Gear: 53 Insp + 35 Multi + 6 Skill && 44 Res + 44 Speed + 6 Skill && (110 Insp + 40 Insp || 110 Multi + 45 Res) + 10 Skill 

### Zarastannil ###
blacksmithing2 = Profession('Blacksmithing')

### Finishing Reagents ###
blacksmithing2.add('Illustrious Insight', 191529, {190456:50}, 'Zarastannil', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Smelthing ###
blacksmithing2.add('Shadowed Alloy', 204994, {190321:1, 190324:1, 204464:1, 189143:5, 190314:2}, 'Zarastannil', 'Crafting Reagent', difficulty=325, insp_percent=35.3, multicraft=6.8, quantity=2, skill=222, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Frostfire Alloy', 190532, {190321:1, 190329:1, 190452:4, 189143:5, 190314:4}, 'Zarastannil', 'Crafting Reagent', difficulty=325, insp_percent=35.3, multicraft=6.8, quantity=2, skill=222, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Infurious Alloy', 190538, {190450:2, 190452:3, 189143:4, 190314:2}, 'Zarastannil', 'Crafting Reagent', difficulty=200, insp_percent=20.3, multicraft=16.8, quantity=2, skill=222, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Primal Molten Alloy', 189543, {190316:1, 190321:1, 190452:4, 189143:5, 190314:4}, 'Zarastannil', 'Crafting Reagent', difficulty=325, insp_percent=35.3, multicraft=6.8, quantity=2, skill=222, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Optional Reagents ###
blacksmithing2.add('Armor Spikes', 191873, {190321:3, 201400:2, 190395:30, 190314:15}, 'Zarastannil', 'Embellishment', difficulty=325, insp_percent=35.3, multicraft=6.8, quantity=1, skill=222, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Stonework ###
blacksmithing2.add('Shadowed Belt Clasp', 205039, {204995:5, 189543:13}, 'Zarastannil', 'Item Enhancement', difficulty=340, insp_percent=36.8, multicraft=3.2, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Primal Razorstone', 191950, {192885:4, 193370:1}, 'Zarastannil', 'Consumable', difficulty=325, insp_percent=35.3, multicraft=8.6, quantity=5, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Primal Whetstone', 191940, {190321:1, 192885:4}, 'Zarastannil', 'Consumable', difficulty=325, insp_percent=35.3, multicraft=8.6, quantity=5, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Primal Weightstone', 191945, {190316:1, 192885:4}, 'Zarastannil', 'Consumable', difficulty=325, insp_percent=35.3, multicraft=8.6, quantity=5, skill=222, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Armor ###
blacksmithing2.add('Allied Chestplate of Generosity', 190519, {206959:1, 193360:1, 190535:10}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Allied Wristguard of Companionship', 190526, {206959:1, 193360:1, 190535:8}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Frostfire Legguards of Preparation', 190523, {206959:1, 193919:1, 193362:1, 190532:16}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Infurious Helm of Vengeance', 190522, {206959:1, 190535:2, 190538:20}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Infurious Warboots of Impunity', 191985, {206959:1, 201402:2, 190535:2, 190538:16}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Breastplate', 190495, {206959:1, 189543:16}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Gauntlets', 190497, {206959:1, 189543:14}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Greatbelt', 190501, {206959:1, 189543:13}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Helm', 190498, {206959:1, 189543:16}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Legplates', 190499, {206959:1, 189543:16}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Pauldrons', 190500, {206959:1, 189543:15}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Sabatons', 190496, {206959:1, 189543:14}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Vambraces', 190502, {206959:1, 189543:14}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Unstable Frostfire Belt', 191623, {206959:1, 193919:1, 193362:1, 190532:13}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Flourishing Dream Helm', 210646, {206959:1, 205413:2, 204995:5, 190535:7, 189543:7}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Explorer's Expert Helm", 201951, {190452:2, 189143:7, 190395:14}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Obsidian Combatant's Draconium Armguards", 204128, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Obsidian Combatant's Draconium Breastplate", 204121, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Obsidian Combatant's Draconium Gauntlets", 204123, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Obsidian Combatant's Draconium Greaves", 204125, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Obsidian Combatant's Draconium Helm", 204124, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Obsidian Combatant's Draconium Pauldrons", 204126, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Obsidian Combatant's Draconium Sabatons", 204122, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Obsidian Combatant's Draconium Waistguard", 204127, {190538:4}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Armguards", 208271, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Breastplate", 208264, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Gauntlets", 208266, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Greaves", 208268, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Helm", 208267, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Pauldrons", 208269, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Sabatons", 208265, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Verdant Combatant's Draconium Waistguard", 208270, {190538:6}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Explorer's Expert Spaulders", 201952, {190452:2, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Explorer's Expert Gauntlets", 201953, {190452:2, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Crimson Combatant's Draconium Armguards", 193609, {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Crimson Combatant's Draconium Breastplate", 193602, {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Crimson Combatant's Draconium Gauntlets", 193604, {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Crimson Combatant's Draconium Greaves", 193606,  {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Crimson Combatant's Draconium Helm", 193605, {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Crimson Combatant's Draconium Pauldrons", 193607, {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Crimson Combatant's Draconium Sabatons", 193603, {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Crimson Combatant's Draconium Waistguard", 193608, {190538:2}, 'Zarastannil', 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Explorer's Expert Greaves", 201954, {190452:2, 189143:7, 190395:14}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Explorer's Expert Clasp", 201955, {190452:2, 189143:4, 190395:10}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Explorer's Plate Chestguard", 189538, {190452:2, 189143:3, 190395:10}, 'Zarastannil', 'Gear (Combat)', difficulty=40, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Explorer's Plate Boots", 189537, {190452:2, 189143:3, 190395:10}, 'Zarastannil', 'Gear (Combat)', difficulty=40, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add("Explorer's Plate Bracers", 189539, {190452:2, 189143:3, 190395:8}, 'Zarastannil', 'Gear (Combat)', difficulty=40, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Shields ###
blacksmithing2.add('Primal Molten Defender', 190503, {206959:1, 189543:15}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Shadowed Impact Buckler', 205168, {206959:1, 205413:1, 204995:5, 189543:15}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Shield of the Hearth', 192081, {206959:1, 193920:1, 201406:1, 189543:16}, 'Zarastannil', 'Gear (Combat)', difficulty=325, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draconium Defender', 201322, {190452:2, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Weapons ###
blacksmithing2.add('Obsidian Seared Claymore', 190514, {206959:2, 190535:8, 189543:5}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Obsidian Seared Crusher', 190516, {206959:2, 190535:7, 189543:7}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Obsidian Seared Facesmasher', 190513, {206959:1, 190535:6, 189543:6}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Obsidian Seared Halberd', 190515, {206959:2, 190535:6, 189543:8}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Obsidian Seared Hexsword', 190511, {206959:1, 190535:6, 189543:6}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Obsidian Seared Invoker', 190517, {206959:2, 190535:7, 189543:5}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Obsidian Seared Runeaxe', 190512, {206959:1, 190535:6, 189543:6}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Obsidian Seared Slicer', 190518, {206959:1, 190535:5, 189543:8}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Greataxe', 190510, {206959:2, 189543:20}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Longsword', 190507, {206959:1, 189543:17}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Mace', 190509, {206959:1, 189543:17}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Shortblade', 190505, {206959:1, 189543:17}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Spellblade', 190506, {206959:1, 189543:17}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Primal Molten Warglaive', 190508, {206959:1, 189543:17}, 'Zarastannil', 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Shadowed Razing Annihilator', 205046, {206959:2, 205413:2, 204995:5, 190535:7, 189543:7}, 'Zarastannil', 'Gear (Combat)', difficulty=300, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draconium Great Mace', 190486, {190452:6, 189143:10, 190395:20}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draocnium Stiletto', 190482, {190452:6, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draconium Great Axe', 201324, {190452:6, 189143:10, 190395:20}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draconium Knuckles', 190485, {190452:6, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draconium Sword', 190484, {190452:5, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draconium Axe', 190487, {190452:5, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
blacksmithing2.add('Draconium Dirk', 190483, {190452:6, 189143:6, 190395:12}, 'Zarastannil', 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Profession Tools and Accessories ###
blacksmithing2.add("Khaz'gorite Blacksmith's Hammer", 191888, {190456:225, 190452:15, 190314:40, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=425, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Khaz'gorite Blacksmith's Toolbox", 191230, {190456:225, 190452:12, 190314:40, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=400, insp_percent=35.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Khaz'gorite Leatherworker's Knife", 191227, {190456:300, 190452:15, 190314:40, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=425, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Khaz'gorite Leatherworker's Toolset", 191229, {190456:300, 190452:12, 190314:40, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=400, insp_percent=35.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Khaz'gorite Needle Set", 191226, {190456:300, 190452:12, 190314:35, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=400, insp_percent=35.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Khaz'gorite Pickaxe", 191223, {190456:300, 190452:15, 190314:45, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=425, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Khaz'gorite Sickle", 191224, {190456:300, 190452:15, 190314:40, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=425, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Khaz'gorite Skinning Knife", 191225, {190456:300, 190452:15, 190314:40, 190395:100}, 'Zarastannil', 'Gear (Profession)', difficulty=425, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Draconium Needle Set', 191239, {190452:3, 189143:4, 190395:10}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=35.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconium Leatherworker's Toolset", 191236, {190452:3, 190395:12, 189143:3}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=35.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconium Leatherworker's Knife", 191238, {190452:3, 189143:3, 190395:12}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconium Blacksmith's Toolbox", 191235, {190452:3, 190395:12, 189143:3}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=35.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconium Skinning Knife", 191240, {190452:4, 189143:3, 190395:10}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Draconium Sickle', 191241, {190452:4, 189143:2, 190395:10}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Draconium Pickaxe', 191242, {190452:4, 189143:2, 190395:10}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconium Blacksmith's Hammer", 191237, {190452:4, 189143:2, 190395:10}, 'Zarastannil', 'Gear (Profession)', difficulty=80, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)

### Consumable Tools ###
blacksmithing2.add("Master's Hammer", 201366, {190452:10, 190535:1, 189543:2, 190532:2}, 'Zarastannil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Heat-Resistant Rescue Ring', 205038, {205413:1, 189543:5, 190532:5, 190538:5, 204995:5}, 'Zarastannil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Sturdy Expedition Shovel', 191304, {190452:2, 190395:10}, 'Zarastannil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Serevite Repair Hammer', 191260, {190452:3, 190395:8}, 'Zarastannil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Serevite Skeleton Key', 191256, {190452:3, 190395:20}, 'Zarastannil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Dragonriding ###
blacksmithing2.add("Prototype Explorer's Barding Framework", 194542, {190452:5, 201399:3, 190395:20, 189543:1, 190535:2}, 'Zarastannil', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Prototype Regal Barding Framework", 194541, {190452:5, 201403:3, 190395:20, 190532:1, 190535:2}, 'Zarastannil', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artisan Curios ###
blacksmithing2.add('Ancient Ceremonial Trident', 203408, {203399:1}, 'Zarastannil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ###
blacksmithing2.add('Obsidian Seared Alloy', 190535, {190324:1, 190321:1, 190452:6, 189143:10, 190314:8}, None, 'Crafting Reagent', difficulty=325, insp_percent=35.3, multicraft=6.8, quantity=2, skill=222, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Armguards", 211570, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Breastplate", 211563, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Gauntlets", 211565, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Greaves", 211567, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Helm", 211566, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Pauldrons", 211568, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Sabatons", 211564, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add("Draconic Combatant's Draconium Waistguard", 211569, {190538:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Alvin the Anvil', 191886, {193920:1, 189543:6, 190532:6, 190538:6}, None, 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
blacksmithing2.add('Black Dragon Touched Hammer', 191228, {190456:400, 193920:1, 190535:7}, None, 'Gear (Profession)', difficulty=450, insp_percent=39.8, multicraft=np.nan, quantity=1, skill=382, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)


In [9]:
blacksmithing_temp = pd.concat((blacksmithing1.get_table(), blacksmithing2.get_table()), ignore_index=True)
blacksmithing_temp['reagents_str'] = blacksmithing_temp['reagents'].apply(str)

drop_rows = list()
blacksmithing_temp = blacksmithing_temp.sort_values(by=['id', 'reagents_str', 'character'])
blacksmithing_temp = blacksmithing_temp.reset_index(drop=True)
for index, row in blacksmithing_temp.iterrows():
    if index == 0:
        prev_row = row
        continue

    if (prev_row['id'] == row['id'] and prev_row['reagents_str']==row['reagents_str'] and 
        (row['character'] is pd.NA or row['character'] is None)):
        drop_rows.append(index)
    prev_row = row
    
blacksmithing_temp = blacksmithing_temp.drop(drop_rows, axis=0)

blacksmithing_temp = blacksmithing_temp.sort_values(by=['id', 'reagents_str', 'character'], 
                                                      ascending=[True, True, False])
blacksmithing_temp = blacksmithing_temp.drop_duplicates(subset=['id', 'skill1', 'reagents_str'], keep='first')

blacksmithing_temp = blacksmithing_temp.sort_values(by=['id', 'reagents_str', 'skill1', 'character'], 
                                                      ascending=[True, True, False, False])
blacksmithing_temp = blacksmithing_temp.drop_duplicates(subset=['id', 'reagents_str'], keep='first')

blacksmithing_temp = blacksmithing_temp.drop('reagents_str', axis=1)

blacksmithing = Profession('blacksmithing')
blacksmithing.set_table(blacksmithing_temp)

### Cooking

In [10]:
# Last Updates:
# Trees: None
# Gear: 53 Speed + 35 Multi + 4 Skill && 44 Res + 66 Speed + 45 Res + 6 Skill 

cooking = Profession('Cooking')

### Ingredients ###
cooking.add('Ooey-Gooey Chocolate', 194902, {197752:3, 197753:6, 197751:4}, 'Trillithia', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Impossibly Sharp Cutting Knife', 197765, {197754:2, 190395:2}, 'Trillithia', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=2, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Salad on the Side', 197764, {197755:2, 191462:1, 191466:1}, 'Trillithia', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=2, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Assorted Exotic Spices', 197757, {197755:2, 197752:1}, 'Trillithia', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity='2-3', skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Pebbled Rock Salts', 197756, {197754:2, 197752:1}, 'Trillithia', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity='2-3', skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Snacks ###
cooking.add('Breakfast of Draconic Champions', 197763, {197743:2, 197744:2, 197746:2, 197745:2, 197747:2, 197748:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Mackeral Snackerel', 197760, {194730:4}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Cheese and Quackers', 197759, {197743:2, 197750:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Probably Protein', 197761, {197741:2, 197752:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Sweet and Sour Clam Chowder', 197762, {197742:2, 197752:2, 197749:5}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Twice-Baked Potato', 197758, {197749:4, 197750:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Beverages ###
cooking.add('Churnbelly Tea', 197772, {194730:1, 194970:1, 197770:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Delicious Dragon Spittle', 197771, {194691:1, 197742:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Slumbering Peacebloom Tea', 207054, {159:1, 2447:2}, 'Trillithia', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Zesty Water', 197770, {197742:1, 159:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Desserts ###
cooking.add('Blubbery Muffin', 197767, {194683:1, 197745:2, 197751:3, 197750:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=3, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Celebratory Cake', 197768, {197745:5, 197751:3, 197753:10, 197752:3}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Snow in a Cone', 197766, {17202:3, 197751:1, 197753:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add("Tasty Hatchling's Treat", 197769, {197741:10, 197745:3, 197751:2, 197753:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=2, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Meat Meals ###
cooking.add('Braised Bruffalon Brisket', 197788, {194967:1, 197746:4, 197757:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Charred Hornswog Steaks', 197774, {197741:4, 197744:4, 197749:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Riverside Picnic', 197789, {197748:4, 194968:1, 197757:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Roast Duck Delight', 197790, {197757:1, 197743:4, 194969:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Salted Meat Mash', 197791, {197741:6, 194730:1, 197756:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Scrambled Basilisk Eggs', 197775, {197741:2, 197745:3, 197750:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Thrice-Spiced Mammoth Kabob', 197776, {197741:4, 197747:4}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Hopefully Healthy', 197777, {197741:3, 197752:4}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Simple Fish Dishes ###
cooking.add('Filet of Fangs', 197779, {194966:2, 197757:2, 197742:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Salt-Baked Fishcake', 197781, {194968:2, 197757:2, 197742:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Seamoth Surprise', 197780, {194967:2, 197757:2, 197742:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Deluxe Fish Dishes ###
cooking.add('Aromatic Seafood Platter', 197783, {194967:1, 194969:1, 197756:3, 194970:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common',  has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Feisty Fish Sticks', 197782, {194969:1, 194966:1, 197756:3, 194970:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Great Cerulean Sea', 197787, {194967:1, 194968:1, 197756:3, 194970:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Revenge, Served Cold', 197785, {194967:1, 194966:1, 197757:3, 194970:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Sizzling Seafood Medley', 197784, {194969:1, 194968:1, 197757:3, 194970:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Thousandbone Tongueslicer', 197786, {194966:1, 194968:1, 197756:3, 194970:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Delicious and Nutritious ###
cooking.add('Fated Fortune Cookie', 197792, {194829:1, 197753:3, 197751:5, 193370:2}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Great Feasts ###
cooking.add("Grand Banquet of the Kalu'ak", 197794, {199344:2, 197790:4, 197784:6, 197785:6, 197747:6, 197768:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add("Grand Banquet of the Kalu'ak", 197794, {200061:2, 197788:4, 197782:6, 197787:6, 197744:5, 197767:3}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add("Grand Banquet of the Kalu'ak", 197794, {199345:2, 197789:4, 197783:6, 197786:6, 197745:6, 197766:3}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add("Yusa's Hearty Stew", 197793, {194970:1, 194966:1, 197747:1, 197749:1, 197757:1}, 'Trillithia', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=2, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artisan Curios ###
cooking.add('Sparkling Spice Pouch', 203409, {203400:1}, 'Trillithia', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ###
cooking.add('Charitable Cheddar', 204848, {3927:5, 138979:2, 162555:1, 3577:1}, None, 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=5, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Rocks on the Rocks', 204845, {197756:3, 190395:5, 189143:1}, None, 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=3, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Firewater Sorbet', 202290, {197766:4, 197751:2}, None, 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=5, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Timely Demise', 197778, {194969:2, 197756:2, 197742:2}, None, 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=4, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Deviously Deviled Eggs', 204072, {197745:4, 197741:2, 197756:3, 197757:3, 197750:10}, None, 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=3, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
cooking.add('Hoard of Draconic Delicacies', 197795, {194970:3, 197788:3, 197790:3, 197756:2, 197757:2}, None, 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=3.2, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

###  Enchanting

In [11]:
# Last Updates:
# Trees: 260/260 && 140/140 && && 175/175
# Gear: 26 Res + 61 Speed + 6 Skill && 61 Insp + 26 Res + 6 Skill && 110 Insp + 40 Insp + 10 Skill

enchanting = Profession('Enchanting')

### Finishing Reagents ###
enchanting.add('Illustrious Insight', 191529, {190456:50}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Cloak Enchantments ###
enchanting.add('Enchant Cloak - Graceful Avoidance', 200031, {194123:8, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Cloak - Homebound Speed', 200032, {194123:8, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Cloak - Regenerative Leech', 200033, {194123:8, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Cloak - Writ of Avoidance', 200034, {194123:12}, 'Linidel', 'Item Enhancement', difficulty=200, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Cloak - Writ of Leech', 200035, {194123:12}, 'Linidel', 'Item Enhancement', difficulty=200, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Cloak - Writ of Speed', 200036, {194123:12}, 'Linidel', 'Item Enhancement', difficulty=200, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Chest Enchantments ###
enchanting.add('Enchant Chest - Accelerated Agility', 200027, {194124:3, 200113:2}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Chest - Reserve of Intellect', 200028, {194124:4, 200113:1}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Chest - Sustained Strength', 200029, {194124:4, 200113:2}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Chest - Waking Stats', 200030, {194123:8, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=350, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Bracer Enchantments ###
enchanting.add('Enchant Bracer - Devotion of Avoidance', 200021, {194123:5, 194124:4}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=36.1, multicraft=np.nan, quantity=1, skill=407, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Bracer - Devotion of Leech', 200022, {194123:5, 194124:4}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=36.1, multicraft=np.nan, quantity=1, skill=407, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Bracer - Devotion of Speed', 200023, {194123:5, 194124:4}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=36.1, multicraft=np.nan, quantity=1, skill=407, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Bracer - Writ of Avoidance', 200024, {194124:1}, 'Linidel', 'Item Enhancement', difficulty=60, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Bracer - Write of Leech', 200025, {194124:1}, 'Linidel', 'Item Enhancement', difficulty=60, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Bracer - Writ of Speed', 200026, {194124:1}, 'Linidel', 'Item Enhancement', difficulty=60, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Boot Enchantments ###
enchanting.add("Enchant Boots - Plainsrunner's Breeze", 200018, {194124:4, 190327:1, 190316:1}, 'Linidel', 'Item Enhancement', difficulty=450, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add("Enchant Boots - Rider's Reassurance", 200019, {194124:4, 190327:1, 190316:1}, 'Linidel', 'Item Enhancement', difficulty=450, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add("Enchant Boots - Watcher's Loam", 200020, {194124:4, 190327:1, 190316:1}, 'Linidel', 'Item Enhancement', difficulty=450, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Ring Enchantments ###
enchanting.add('Enchant Ring - Devotion of Critical Strike', 200037, {194123:5, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=36.1, multicraft=np.nan, quantity=1, skill=407, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Ring - Devotion of Haste', 200038, {194123:5, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=36.1, multicraft=np.nan, quantity=1, skill=407, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Ring - Devotion of Mastery', 200039, {194123:5, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=36.1, multicraft=np.nan, quantity=1, skill=407, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Ring - Devotion of Versatility', 200040, {194123:5, 194124:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=36.1, multicraft=np.nan, quantity=1, skill=407, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Ring - Writ of Critical Strike', 200041, {194123:3}, 'Linidel', 'Item Enhancement', difficulty=40, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Ring - Writ of Haste', 200042, {194123:3}, 'Linidel', 'Item Enhancement', difficulty=40, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Ring - Writ of Mastery', 200043, {194123:3}, 'Linidel', 'Item Enhancement', difficulty=40, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Ring - Writ of Versatility', 200044, {194123:3}, 'Linidel', 'Item Enhancement', difficulty=40, insp_percent=35.1, multicraft=np.nan, quantity=1, skill=407, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Weapon Enchantments ###
enchanting.add('Enchant Weapon - Burning Devotion', 200050, {194124:5, 200113:4, 190321:6, 204106:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=382, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Earthen Devotion', 200052, {194124:5, 200113:4, 190316:6, 204106:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=382, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Frozen Devotion', 200056, {194124:5, 200113:4, 190329:6, 204106:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=382, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Shadowflame Wreathe', 204623, {194124:4, 200113:4, 204464:1, 190321:4}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Sophic Devotion', 200054, {194124:5, 200113:4, 190324:4, 204106:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=382, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Spore Tender', 204615, {194124:4, 200113:4, 204460:150, 190316:4}, 'Linidel', 'Item Enchancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Wafting Devotion', 200058, {194124:5, 200113:4, 190327:6, 204106:3}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=382, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Burning Writ', 200051, {194123:15, 200113:2, 190321:4}, 'Linidel', 'Item Enhancement', difficulty=300, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Earthen Writ', 200053, {194123:15, 200113:2, 190316:4}, 'Linidel', 'Item Enhancement', difficulty=300, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Frozen Writ', 200057, {194123:15, 200113:2, 190329:4}, 'Linidel', 'Item Enhancement', difficulty=300, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Sophic Writ', 200055, {194123:15, 200113:2, 190324:3}, 'Linidel', 'Item Enhancement', difficulty=300, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Weapon - Wafting Writ', 200059, {194123:15, 200113:2, 190327:4}, 'Linidel', 'Item Enhancement', difficulty=300, insp_percent=34.6, multicraft=np.nan, quantity=1, skill=382, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Profession Tool Enchantments ###
enchanting.add('Enchant Tool - Draconic Deftness', 200045, {194124:4, 200113:2, 201401:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Tool - Draconic Finesse', 200046, {194124:4, 200113:2, 201401:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Tool - Draconic Inspiration', 200047, {194124:4, 200113:2, 201401:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Tool - Draconic Perception', 200048, {194124:4, 200113:2, 201401:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchant Tool - Draconic Resourcefulness', 200049, {194124:4, 200113:2, 201401:3}, 'Linidel', 'Item Enhancement', difficulty=400, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=392, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Rods and Wands ###
enchanting.add('Spore Keeper\'s Baton', 204401, {206959:1, 200113:3, 204460:200, 194864:2, 192871:1}, 'Linidel', 'Gear (Combat)', difficulty=320, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=267, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
enchanting.add('Torch of Primal Awakening', 200642, {206959:1, 194124:2, 200113:3, 194864:2, 189543:2}, 'Linidel', 'Gear (Combat)', difficulty=265, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=267, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
enchanting.add("Runed Khaz'gorite Rod", 198716, {190456:300, 194124:5, 200113:1, 190314:4, 194864:2}, 'Linidel', 'Gear (Profession)', difficulty=350, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Runed Draconium Rod', 198715, {194123:4, 189143:3, 191472:2}, 'Linidel', 'Gear (Profession)', difficulty=80, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=267, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Writhebark Wand', 200641, {194123:6, 191472:2}, 'Linidel', 'Gear (Combat)', difficulty=60, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
enchanting.add('Runed Serevite Rod', 201601, {194123:3, 201584:1}, 'Linidel', 'Gear (Profession)', difficulty=40, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=267, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)

### Illusory Goods ###
enchanting.add('Illusion: Primal Air', 200883, {200113:2, 190327:20}, 'Linidel', 'Cosmetic', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Khadgar\'s Disenchanting Rod', 200469, {194123:12, 194124:6, 200113:3}, 'Linidel', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan,quantity=1,  skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusory Adornment: Order', 201319, {194123:2, 190322:2}, 'Linidel', 'Cosmetic', difficulty=275, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=252, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusory Adornment: Air', 201316, {194123:2, 190326:2}, 'Linidel', 'Cosmetic', difficulty=275, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=252, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusory Adornment: Earth', 201313, {194123:2, 190315:2}, 'Linidel', 'Cosmetic', difficulty=275, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=252, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusory Adornment: Fire', 200616, {194123:2, 190320:2}, 'Linidel', 'Cosmetic', difficulty=275, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=252, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusory Adornment: Frost', 201310, {194123:2, 190328:2}, 'Linidel', 'Cosmetic', difficulty=275, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=252, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusory Adornment: Spores', 204567, {194123:2, 204460:20, 190315:1}, 'Linidel', 'Cosmetic', difficulty=275, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=252, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Scepter of Spectacle: Order', 201158, {194123:1, 190322:3, 191472:1}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Scepter of Spectacle: Air', 201161, {194123:1, 190326:3, 191472:1}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Scepter of Spectacle: Frost', 200535, {194123:1, 190328:3, 191472:1}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Scepter of Spectacle: Earth', 201160, {194123:1, 190315:3, 191472:1}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Scepter of Spectacle: Fire', 200472, {194123:1, 190320:3, 191472:1}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Magical Merchandise ###
enchanting.add('Sophic Amalgamation', 200479, {200113:3, 190324:3}, 'Linidel', 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Infusions of Power ###
enchanting.add('Enchanted Aspect\'s Dreaming Crest', 204697, {194123:80, 194124:15, 200113:4, 204463:2, 208393:1}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Aspect\'s Shadowflame Crest', 204697, {194123:80, 194124:15, 200113:4, 204463:2, 204194:4}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Wyrm\'s Dreaming Crest', 204682, {194123:60, 194124:10, 200113:2, 204463:1, 208394:1}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Wyrm\'s Shadowflame Crest', 204682, {194123:60, 194124:10, 200113:2, 204463:1, 204196:3}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Whelpling\'s Dreaming Crest', 204681, {194123:30, 194124:5, 200113:1, 208395:1}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Whelpling\'s Shadowflame Crest', 204681, {194123:30, 194124:5, 200113:1, 204193:1}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Titan Training Matrix V', 204673, {194123:10, 194124:1, 205263:1}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artisan Curios ###
enchanting.add('Glowing Crystal Bookmark', 203410, {203401:1}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=0.0, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ###
enchanting.add('Dreaming Devotion', 207087, {194124:4, 200113:4, 208212:1, 190316:1, 190324:1, 201406:2}, None, 'Item Enhancement', difficulty=425, insp_percent=35.6, multicraft=np.nan, quantity=1, skill=397, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusion: Primal Earth', 200905, {200113:2, 190316:20}, None, 'Cosmetic', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusion: Primal Fire', 200906, {200113:2, 190321:20}, None, 'Cosmetic', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusion: Primal Frost', 200907, {200113:2, 190329:20}, None, 'Cosmetic', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusion: Primal Mastery', 200470, {200113:5, 190327:5, 190316:5, 190321:5, 190329:5}, None, 'Cosmetic', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Primal Invocation Extract', 200634, {190327:1, 190316:1, 190321:1, 190329:1, 190324:1}, None, 'Other', difficulty=300, insp_percent=29.6, multicraft=0.0, quantity=1, skill=312, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Illusory Adornment: Dreams', 206969, {194123:2, 190315:1, 190322:1}, None, 'Cosmetic', difficulty=290, insp_percent=29.6, multicraft=np.nan, quantity=1, skill=267, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Aspect\'s Awakened Crest', 211519, {194123:80, 194124:15, 200113:4, 204463:2, 211522:1}, None, 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Wyrm\'s Awakened Crest', 211518, {194123:60, 194124:10, 200113:2, 204463:1, 211521:1}, None, 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
enchanting.add('Enchanted Whelpling\'s Awakened Crest', 211520, {194123:30, 194124:5, 200113:1, 211523:1}, None, 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)


### Engineering

In [12]:
# Last Updates:
# Trees: 150/150 && 130/130 && 135/135 && 110/110
# Gear: 53 Insp + 35 Multi + 6 Skill && 44 Res + 44 Speed + 6 Skill && (110 Multi + 45 Res || 110 Insp + 40 Insp) + 10 Skill

engineering = Profession('Engineering')

### Arclight Capacitor ### 
engineering.add("Arclight Capacitor", 198197, {190324: 1, 198188: 2, 198194: 1, 190314: 2}, "Trillithia", 'Crafting Reagent', difficulty=350, insp_percent=29.8, multicraft=7.7, quantity=1, skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Reinforced Machine Chassis", 198200, {190316: 1, 198185: 4, 198188: 1, 198194: 2}, "Trillithia", 'Crafting Reagent', difficulty=300, insp_percent=29.8, multicraft=7.7, quantity=1, skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Assorted Safety Fuses", 198203, {193922: 3, 198185: 3, 198188: 1, 198194: 1}, "Trillithia", 'Crafting Reagent', difficulty=250, insp_percent=14.8, multicraft=17.7, quantity="2-3", skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Everburning Blasting Powder", 198191, {190320: 2, 190315: 1, 189143: 1}, "Trillithia", 'Crafting Reagent', difficulty=200, insp_percent=14.8, multicraft=17.7, quantity="1-2", skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Greased-Up Gears", 198194, {190320: 3, 198185: 2, 189143: 4}, "Trillithia", 'Crafting Reagent', difficulty=250, insp_percent=14.8, multicraft=17.7, quantity="1-2", skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Shock-Spring Coil", 198188, {190315: 2, 198185: 2, 190395: 5}, "Trillithia", 'Crafting Reagent', difficulty=150, insp_percent=14.8, multicraft=17.7, quantity="1-2", skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Handful of Serevite Bolts", 198185, {190395: 4}, "Trillithia", 'Crafting Reagent', difficulty=50, insp_percent=14.8, multicraft=17.7, quantity="2-3", skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Finishing Reagents ### 
engineering.add("Overcharged Overclocker", 198221, {19032: 5, 198185: 2, 198188: 3}, "Trillithia", 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=2, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Illustrious Insight", 191529, {190456: 50}, "Trillithia", 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Haphazardly Tethered Wires", 198218, {190315: 4, 198185: 3}, "Trillithia", 'Crafting Reagent', difficulty=250, insp_percent=19.8, multicraft=17.7, quantity=2, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Optional Reagents ### 
engineering.add("Magazine of Healing Darts", 198258, {198197: 1, 191380: 10, 198191: 3, 198200: 1}, "Trillithia", 'Embellishment', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity=' rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Tinkers ### 
engineering.add("Tinker: Alarm-O-Turret", 198291, {190450: 2, 198200: 1, 198197: 1, 198194: 3, 198188: 3}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker: Arclight Vital Correctors", 201409, {190324: 2, 198200: 1, 198197: 2, 198194: 4, 198188: 3}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker: Polarity Amplifier", 199190, {190450: 1, 198200: 1, 198194: 3, 198197: 1, 198185: 8}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker: Shadowflame Rockets", 205016, {204464: 1, 204463: 1, 198188: 2, 198194: 4, 198200: 1, 198197: 1}, 'Trillithia', 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker: Supercollide-O-Tron", 198303, {190321: 5, 190324: 3, 198188: 3, 198194: 10, 198200: 1, 198197: 3}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker: Ground Circuitry", 198306, {198200: 1, 198185: 4, 198197: 2, 198194: 2}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker: Plane Displacer", 198300, {198188: 1, 198200: 1, 191395: 1}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Goggles ### 
engineering.add("Battle-Ready Binoculars", 198326, {206959: 1, 190324: 2, 192878: 2, 190535: 2, 198197: 2, 198200: 1}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)
engineering.add("Lightweight Ocular Lenses", 198323, {206959: 1, 190324: 2, 192878: 2, 198197: 2, 198200: 1, 193931: 2}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)
engineering.add("Oscillating Wilderness Opticals", 198325, {206959: 1, 190324: 2, 193247: 4, 192878: 2, 198197: 2, 198200: 1}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)
engineering.add("Peripheral Vision Projectors", 198324, {206959: 1, 190324: 2, 193228: 4, 192878: 2, 198197: 2, 198200: 1}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)
engineering.add("Obsidian Combatant's Cloth Goggles", 205278, {193934: 5, 192878: 2, 198185: 2}, 'Trillithia', 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Obsidian Combatant's Leather Goggles", 205279, {193238: 3, 192878: 2, 198185: 2}, "Trillithia", 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Obsidian Combatant's Mail Goggles", 205280, {193250: 3, 192878: 2, 198185: 2}, "Trillithia", 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Obsidian Combatant's Plate Goggles", 205281, {190538: 5, 192878: 2, 198185: 2}, 'Trillithia', 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Verdant Combatant's Cloth Goggles", 208313, {193934: 6, 192878: 2, 198185: 4}, 'Trillithia', 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Verdant Combatant's Leather Goggles", 208314, {193238: 4, 192878: 2, 198185: 4}, "Trillithia", 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Verdant Combatant's Mail Goggles", 208315, {193250: 4, 192878: 2, 198185: 4}, 'Trillithia', 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Verdant Combatant's Plate Goggles", 208316, {190538: 6, 192878: 2, 198185: 4}, 'Trillithia', 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Deadline Deadeyes", 198330, {201832: 2, 198188: 2, 198185: 2, 198194: 1}, "Trillithia", 'Gear (Combat)', difficulty=40, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
engineering.add("Milestone Magnifiers", 198329, {201832: 2, 198188: 2, 198185: 2, 198194: 1}, "Trillithia", 'Gear (Combat)', difficulty=40, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
engineering.add("Quality-Assured Optics", 198328, {201832: 2, 198185: 2, 198194: 1, 198188: 2}, "Trillithia", 'Gear (Combat)', difficulty=40, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
engineering.add("Sentry's Stabilized Specs", 198331, {201832: 2, 198185: 2, 198194: 1, 198188: 2}, "Trillithia", 'Gear (Combat)', difficulty=40, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)

### Armor ### 
engineering.add("Complicated Cuffs", 198332, {206959: 1, 193224: 5, 198200: 1, 198194: 3, 198197: 3}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)
engineering.add("Difficult Wrist Protectors", 198333, {206959: 1, 190535: 2, 198200: 1, 198194: 3, 198197: 3}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)
engineering.add("Needlessly Complex Wristguards", 198327, {206959: 1, 193218: 5, 198200: 1, 198194: 3, 198197: 3}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)
engineering.add("Overengineered Sleeve Extenders", 198322, {206959: 1, 193931: 4, 198200: 1, 198194: 2, 198197: 2}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=True, has_crest_slot=True)

### Weapons ### 
engineering.add("Ol' Smoky", 198335, {206959: 2, 190535: 4, 198200: 1, 198191: 8, 198197: 2}, "Trillithia", 'Gear (Combat)', difficulty=320, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=302, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
engineering.add("P.E.W. x2", 198334, {190320: 1, 198185: 5, 198191: 5, 190395: 5}, "Trillithia", 'Gear (Combat)', difficulty=60, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=302, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Cogwheels ### 
engineering.add("Meticulously-Tuned Gear", 198238, {198185: 4, 198194: 1, 192848: 1}, "Trillithia", 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("One-Size-Fits-All Gear", 198309, {198185: 4, 198194: 1, 192842: 1}, "Trillithia", 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Rapidly Ticking Gear", 198233, {198185: 4, 198194: 1, 192845: 1}, "Trillithia", 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Razor-Sharp Gear", 198176, {198185: 4, 198194: 1, 192839: 1}, "Trillithia", 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Scopes & Ammo ### 
engineering.add("High Intensity Thermal Scanner", 198318, {192878: 1, 198194: 3, 198197: 2, 198200: 1}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=11.4, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Completely Safe Rockets", 198162, {198185: 2, 198191: 4}, "Trillithia", 'Consumable', difficulty=425, insp_percent=31.8, multicraft=11.4, quantity=2, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Endless Stack of Needles", 198165, {198185: 2, 198188: 1, 198194: 1}, "Trillithia", 'Consumable', difficulty=425, insp_percent=31.8, multicraft=11.4, quantity=2, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Gyroscopic Kaleidoscope", 198312, {192878: 1, 198185: 2, 198194: 6, 198197: 2}, "Trillithia", 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=11.4, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Fireworks ### 
engineering.add("Black Fireflight", 198549, {193922: 1, 198191: 2}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity="2-3", skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Blue Fireflight", 198222, {193922: 1, 198191: 2}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity="2-3", skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Bundle of Fireworks", 198172, {190320: 3, 198191: 2, 193228: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Green Fireflight", 198224, {198191: 2, 193215: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity="2-3", skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Red Fireflight", 198223, {193208: 1, 198191: 2}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity="2-3", skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Bronze Fireflight", 198550, {198191: 2, 192851: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity="2-3", skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Explosives ### 
engineering.add("Suspiciously Silent Crate", 198171, {190321: 1, 198191: 10, 198188: 1, 198185: 3, 198203: 2}, "Trillithia", 'Consumable', difficulty=450, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=327, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Suspiciously Ticking Crate", 198168, {190321: 1, 198191: 10, 198188: 1, 198185: 3}, "Trillithia", 'Consumable', difficulty=450, insp_percent=31.8, multicraft=np.nan, quantity=1, skill=322, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("I.W.I.N. Button Mk10", 198159, {198185: 1, 198197: 1, 198188: 1, 198280: 1}, "Trillithia", 'Consumable', difficulty=300, insp_percent=16.8, multicraft=21.4, quantity="1-2", skill=322, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Polarity Bomb", 204826, {190315: 1, 198191: 6, 198188: 1, 198185: 4}, "Trillithia", 'Consumable', difficulty=250, insp_percent=16.8, multicraft=21.4, quantity="2-3", skill=322, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Gravitational Displacer", 198230, {198191: 4, 198188: 1, 198185: 2, 191398: 1}, "Trillithia", 'Consumable', difficulty=350, insp_percent=31.8, multicraft=11.4, quantity="2-3", skill=322, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Grease Grenade", 198294, {190330: 1, 198185: 4, 198191: 6}, "Trillithia", 'Consumable', difficulty=300, insp_percent=16.8, multicraft=21.4, quantity="2-3", skill=322, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Sticky Warp Grenade", 198283, {190330: 1, 198191: 3, 198188: 2, 198185: 6}, "Trillithia", 'Consumable', difficulty=450, insp_percent=31.8, multicraft=11.4, quantity="2-3", skill=322, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Primal Deconstruction Charge", 198280, {190320: 1, 198191: 4, 198188: 1, 198185: 4}, "Trillithia", 'Consumable', difficulty=250, insp_percent=16.8, multicraft=21.4, quantity="2-3", skill=322, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Creature Combustion Cansiter", 198241, {198191: 6, 198188: 1, 198185: 4}, "Trillithia", 'Consumable', difficulty=250, insp_percent=16.8, multicraft=21.4, quantity="2-3", skill=322, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### EZ-Thro ### 
engineering.add("EZ-Thro Polarity Bomb", 204829, {190320: 1, 198191: 6, 198188: 1, 198185: 4, 198203: 1}, "Trillithia", 'Consumable', difficulty=250, insp_percent=16.8, multicraft=21.4, quantity="2-3", skill=327, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Devices ### 
engineering.add("Atomic Recalibrator", 198173, {198185: 20, 198188: 6, 198200: 2, 198197: 2, 198194: 4}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Centralized Precipitation Emitter", 198264, {193919: 3, 193921: 3, 198185: 20, 198188: 3, 198200: 1, 198197: 4, 192887: 1}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Element-Infused Rocket Helmet", 192443, {190321: 10, 190327: 5, 198230: 15, 198191: 20, 198200: 1, 198197: 2, 198185: 5}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Environmental Emulator", 198206, {193362: 3, 193920: 3, 198185: 20, 198188: 3, 198200: 1, 198197: 4, 192887: 1}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Giggle Goggles", 198227, {192878: 2, 198188: 3, 198200: 1, 198197: 2, 198185: 6, 192862: 2}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Mallard Mortar", 204818, {198185: 60, 198188: 6, 198194: 8, 198200: 2, 198197: 2}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add('Portable Party Platter', 207092, {197768: 10, 193919: 5, 198197: 6, 198194: 12, 198185: 60, 198200: 5}, 'Trillithia', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("S.A.V.I.O.R.", 198277, {190324: 1, 198185: 20, 198197: 1, 198188: 3, 198194: 2}, "Trillithia", 'Other', difficulty=350, insp_percent=29.8, multicraft=6.8, quantity=2, skill=242, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Zapthrottle Soul Inhaler", 199414, {198191: 3, 198197: 1, 198200: 1, 198185: 6, 198188: 2, 198194: 3}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("H.E.L.P.", 201930, {190450: 3, 137642: 15, 198200: 1, 198191: 4, 198185: 6}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker Removal Kit", 202087, {198185: 3, 198188: 2, 189143: 2}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Wyrmhole Generator: Dragon Isles", 198156, {190324: 3, 198185: 2, 198188: 1, 198194: 2, 198200: 1, 198197: 3}, "Trillithia", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Portable Alchemist's Lab Bench", 198265, {190326: 1, 190328: 1, 198194: 1, 198185: 3, 191498: 3, 191476: 5}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Portable Tinker's Workbench", 198268, {198185: 2, 192878: 1, 198194: 1, 198188: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Neural Silencer Mk3", 198247, {193922: 1, 198185: 4, 198188: 2, 198194: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity="2-3", skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Profession Equipment ### 
engineering.add("Khaz'gorite Brainwave Amplifier", 198205, {190456: 225, 192878: 2, 190314: 10, 198188: 2, 198194: 3, 198197: 4}, "Trillithia", 'Gear (Profession)', difficulty=425, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=297, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Khaz'gorite Delver's Helmet", 198244, {190456: 225, 192878: 1, 190535: 4, 198197: 2}, "Trillithia", 'Gear (Profession)', difficulty=425, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=297, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Khaz'gorite Encased Samophlange", 198246, {190456: 300, 190314: 10, 198188: 2, 198194: 3, 198197: 5}, "Trillithia", 'Gear (Profession)', difficulty=400, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=297, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Khaz'gorite Fisherfriend", 198226, {190314: 10, 198194: 4, 198197: 2, 198200: 1}, "Trillithia", 'Gear (Profession)', difficulty=425, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=297, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Lapidary's Khaz'gorite Clamps", 198235, {190456: 300, 190314: 20, 198197: 2, 198194: 5}, "Trillithia", 'Gear (Profession)', difficulty=400, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=297, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Spring-Loaded Khaz'gorite Fabric Cutters", 194126, {190456: 300, 190535: 4, 198197: 2, 198188: 2}, "Trillithia", 'Gear (Profession)', difficulty=400, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=297, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Bottomless Mireslush Ore Satchel", 198263, {190456: 225, 193231: 6, 190532: 5}, "Trillithia", 'Gear (Profession)', difficulty=425, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=297, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Bottomless Stonecrust Ore Satchel", 198262, {189143: 5, 198185: 2}, "Trillithia", 'Gear (Profession)', difficulty=80, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Draconium Brainwave Amplifier", 198204, {201832: 1, 198185: 2, 198188: 2, 189143: 2}, "Trillithia", 'Gear (Profession)', difficulty=80, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Draconium Delver's Helmet", 198243, {201832: 1, 198185: 2, 189143: 2}, "Trillithia", 'Gear (Profession)', difficulty=80, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Draconium Encased Samophlange", 198245, {198185: 2, 198188: 2, 198194: 1, 189143: 2}, "Trillithia", 'Gear (Profession)', difficulty=80, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Draconium Fisherfriend", 198225, {198185: 2, 198194: 1, 198188: 2, 189143: 3}, "Trillithia", 'Gear (Profession)', difficulty=80, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Lapidary's Draconium Clamps", 198234, {198185: 2, 198194: 1, 189143: 3}, "Trillithia", 'Gear (Profession)', difficulty=80, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Spring-Loaded Draconium Fabric Cutters", 194125, {198185: 3, 198188: 3, 189143: 4}, "Trillithia", 'Gear (Profession)', difficulty=80, insp_percent=19.8, multicraft=np.nan, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)

### Robotics ### 
engineering.add("Quack-E", 193572, {193053: 60, 198796: 1, 198200: 1, 192858: 2, 198194: 2, 198188: 1, 198185: 4}, "Trillithia", 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("D.U.C.K.O.Y.", 198182, {193053: 6, 193208: 1, 198185: 1, 198191: 2}, "Trillithia", 'Consumable', difficulty=350, insp_percent=29.8, multicraft=6.8, quantity=1, skill=242, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artisan Curios ### 
engineering.add("Gnomish Voicebox", 203411, {203402: 1}, "Trillithia", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=16.8, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ### 
engineering.add("Calibrated Safety Switch", 198255, {198197: 1, 198188: 2, 198194: 1, 198200: 1}, None, 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Critical Failure Prevention Unit", 198261, {198197: 1, 198280: 2, 198188: 3, 198200: 1}, None, 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Spring-Loaded Capacitor Casing", 198621, {198197: 1, 198188: 2, 198185: 10, 198200: 1}, None, 'Crafting Reagent', difficulty=425, insp_percent=34.8, multicraft=7.7, quantity=1, skill=297, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Tinker: Breath of Neltharion", 198297, {190321: 3, 198200: 1, 198191: 8, 198185: 5, 198194: 4}, None, 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=6.8, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("Draconic Combatant's Cloth Goggles", 211612, {193934: 6, 192878: 2, 198185: 4}, None, 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Draconic Combatant's Leather Goggles", 211613, {193238: 4, 192878: 2, 198185: 4}, None, 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Draconic Combatant's Mail Goggles", 211614, {193250: 4, 192878: 2, 198185: 4}, None, 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Draconic Combatant's Plate Goggles", 211615, {190538: 6, 192878: 2, 198185: 4}, None, 'Gear (Combat)', difficulty=120, insp_percent=20.8, multicraft=np.nan, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=False)
engineering.add("Projectile Propulsion Pinion", 198315, {192878: 1, 198194: 3, 198197: 2, 198200: 1}, None, 'Item Enhancement', difficulty=425, insp_percent=31.8, multicraft=11.4, quantity=1, skill=302, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("EZ-Thro Creature Combustion Canister", 198209, {198191: 6, 198188: 1, 198185: 4, 198203: 1}, None, 'Consumable', difficulty=250, insp_percent=16.8, multicraft=21.4, quantity="2-3", skill=327, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("EZ-Thro Gravitational Displacer", 198215, {190330: 1, 198191: 4, 198188: 1, 198185: 2, 191398: 1, 198203: 1}, None, 'Consumable', difficulty=350, insp_percent=31.8, multicraft=11.4, quantity="2-3", skill=327, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add("EZ-Thro Grease Grenade", 199195, {190330: 1, 198203: 1, 198185: 4, 198191: 6}, None, 'Consumable', difficulty=300, insp_percent=16.8, multicraft=21.4, quantity="2-3", skill=327, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
engineering.add('EZ-Thro Primal Deconstruction Charge', 198212, {190320:1, 198191:4, 198188:1, 198185:4, 198203:1}, None, 'Consumable', difficulty=250, insp_percent=16.8, multicraft=21.4, quantity='2-3', skill=327, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)


### Inscription

In [13]:
# Last Updates:
# Trees: 140/140 && 330/330 && 210/210
# Gear: 53 Insp + 35 Multi + 6 Skill && 35 Res + 53 Speed + 6 Skill && (110 Multi + 45 Res || 110 Insp + 40 Insp) + 10 Skill 

inscription = Profession('Inscription')

### Inks ###
inscription.add('Cosmic Ink', 194756, {198487:1, 190329:1, 194858:1, 194760:2, 194864:1}, 'Mellasona', 'Crafting Reagent', difficulty=300, insp_percent=34.8, multicraft=5.0, quantity=2, skill=202, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Burnished Ink', 194855, {198487:1, 194751:2, 194852:2, 194858:2}, 'Mellasona', 'Crafting Reagent', difficulty=260, insp_percent=34.8, multicraft=5.0, quantity=2, skill=202, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Blazing Ink', 194846, {198487:1, 198423:2, 198420:1, 191476:1}, 'Mellasona', 'Crafting Reagent', difficulty=80, insp_percent=19.8, multicraft=15.0, quantity=2, skill=202, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Flourishing Ink', 194852, {198487:1, 198423:2, 198417:1, 191476:1}, 'Mellasona', 'Crafting Reagent', difficulty=80, insp_percent=19.8, multicraft=15.0, quantity=2, skill=202, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Serene Ink', 194858, {198487:1, 198423:2, 198414:1, 191476:1}, 'Mellasona', 'Crafting Reagent', difficulty=80, insp_percent=19.8, multicraft=15.0, quantity=2, skill=202, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Reagents ###
inscription.add('Illustrious Insight', 191529, {190456:50}, 'Mellasona', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Runed Writhebark', 194864, {190326:5, 191472:5, 194852:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=260, insp_percent=33.3, multicraft=3.2, quantity=1, skill=182, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Chilled Rune', 194768, {190328:5, 194858:1}, 'Mellasona', 'Crafting Reagent', difficulty=260, insp_percent=33.3, multicraft=3.2, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Missives ###
inscription.add('Draconic Missive of the Aurora', 194569, {194784:1, 194859:1, 194858:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of the Feverflare', 194566, {194784:1, 194859:1, 194751:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of the Fireflash', 192552, {194784:1, 194859:1, 194751:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of the Harmonious', 194575, {194784:1, 194859:1, 194858:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of the Peerless', 194578, {194784:1, 194859:1, 194852:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of the Quickblade', 194572, {194784:1, 194859:1, 194852:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Crafting Tool Missives ###
inscription.add('Draconic Missive of Crafting Speed', 200573, {194784:1, 194852:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=212, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of Inspiration', 198536, {194784:1, 194751:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of Multicraft', 200570, {194784:1, 194852:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of Resourcefulness', 200567, {194784:1, 194751:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Gathering Tool Missives ###
inscription.add('Draconic Missive of Deftness', 200582, {194784:1, 194858:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of Finesse', 200576, {194784:1, 194852:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Missive of Perception', 200579, {194784:1, 194858:1, 194859:1}, 'Mellasona', 'Crafting Reagent', difficulty=275, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Runes and Sigils ###
inscription.add('Vantus Rune: Aberrus, the Shadowed Crucible', 204860, {194760:2, 194859:1}, 'Mellasona', 'Consumable', difficulty=360, insp_percent=37.8, multicraft=4.1, quantity=1, skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Vantus Rune: Amidrassil, the Dream\'s Hope', 210249, {194855:2, 194768:1}, 'Mellasona', 'Consumable', difficulty=360, insp_percent=37.8, multicraft=14.1, quantity=1, skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Vantus Rune: Vault of the Incarnates', 198493, {194760:2, 194859:1}, 'Mellasona', 'Consumable', difficulty=360, insp_percent=37.8, multicraft=4.1, quantity=1, skill=252, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Emberscale Sigil', 199056, {190321:1, 194751:1, 194760:2}, 'Mellasona', 'Crafting Reagent', difficulty=300, insp_percent=35.8, multicraft=3.2, quantity=1, skill=242, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Jetscale Sigil', 199058, {190450:1, 194756:1}, 'Mellasona', 'Crafting Reagent', difficulty=300, insp_percent=35.8, multicraft=3.2, quantity=1, skill=242, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Sagescale Sigil', 199060, {190316:1, 194852:1, 194760:2}, 'Mellasona', 'Crafting Reagent', difficulty=300, insp_percent=35.8, multicraft=3.2, quantity=1, skill=242, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Azurescale Sigil', 199052, {190329:1, 194858:1, 194760:2}, 'Mellasona', 'Crafting Reagent', difficulty=300, insp_percent=35.8, multicraft=3.2, quantity=1, skill=242, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Bronzescale Sigil', 199054, {190324:1, 194760:2}, 'Mellasona', 'Crafting Reagent', difficulty=300, insp_percent=35.8, multicraft=3.2, quantity=1, skill=242, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Buzzing Rune', 194823, {194859:1}, 'Mellasona', 'Consumable', difficulty=300, insp_percent=37.8, multicraft=4.1, quantity=2, skill=252, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Chirping Rune', 194826, {194859:1}, 'Mellasona', 'Consumable', difficulty=300, insp_percent=37.8, multicraft=4.1, quantity=2, skill=252, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Hissing Rune', 204973, {194859:1}, 'Mellasona', 'Consumable', difficulty=300, insp_percent=37.8, multicraft=4.1, quantity=2, skill=252, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Howling Rune', 194820, {194859:1}, 'Mellasona', 'Consumable', difficulty=300, insp_percent=37.8, multicraft=4.1, quantity=2, skill=252, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Trinkets ###
inscription.add('Darkmoon Deck Box: Dance', 198478, {206959:1, 198088:1, 190327:10, 191472:10, 194756:12}, 'Mellasona', 'Gear (Combat)', difficulty=360, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=307, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=True)
inscription.add('Darkmoon Deck Box: Inferno', 194872, {206959:1, 198086:1, 190321:10, 191472:10, 194756:12}, 'Mellasona', 'Gear (Combat)', difficulty=360, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=307, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=True)
inscription.add('Darkmoon Deck Box: Rime', 198477, {206959:1, 198087:1, 190329:10, 191472:10, 194756:12}, 'Mellasona', 'Gear (Combat)', difficulty=360, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=307, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=True)
inscription.add('Darkmoon Deck Box: Watcher', 198481, {206959:1, 198089:1, 190316:10, 191472:10, 194756:12}, 'Mellasona', 'Gear (Combat)', difficulty=360, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=307, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=True, has_crest_slot=True)
inscription.add("Obsidian Combatant's Emblem", 204166, {190450:3, 194859:1, 194852:6}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Obsidian Combatant's Insignia of Alacrity", 204165, {190450:3, 194859:1, 194751:6}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Obsidian Combatant's Medallion", 204164, {190450:3, 194859:1, 194858:6}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Verdant Combatant's Emblem", 204166, {190450:4, 194768:1, 194852:10}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Verdant Combatant's Insignia of Alacrity", 204165, {190450:4, 194768:1, 194751:10}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Verdant Combatant's Medallion", 204164, {190450:4, 194768:1, 194858:10}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Crimson Combatant's Emblem", 203377, {190450:3, 194859:1, 194852:3}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Crimson Combatant's Insignia of Alacrity", 203376, {190450:3, 194859:1, 194751:3}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Crimson Combatant's Medallion", 203375, {190450:3, 194859:1, 194858:3}, 'Mellasona', 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Weapons ###
inscription.add('Crackling Codex of the Isles', 194879, {206959:1, 190327:5, 194784:20, 194756:10, 194864:5, 194859:4}, 'Mellasona', 'Gear (Combat)', difficulty=300, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=292, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
inscription.add('Illuminating Pillar of the Isles', 194898, {206959:2, 194756:6, 194864:10, 194859:8}, 'Mellasona', 'Gear (Combat)', difficulty=300, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=292, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
inscription.add('Kinetic Pillar of the Isles', 194897, {206959:2, 194756:6, 194864:10, 194859:8}, 'Mellasona', 'Gear (Combat)', difficulty=300, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=292, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
inscription.add("Core Explorer's Compendium", 194877, {194784:20, 190315:5, 194751:1, 194859:1}, 'Mellasona', 'Gear (Combat)', difficulty=60, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
inscription.add("Overseer's Writhebark Stave", 194893, {194751:1, 194864:1}, 'Mellasona', 'Gear (Combat)', difficulty=60, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
inscription.add("Pioneer's Writhebark Stave", 194892, {194858:1, 194864:1}, 'Mellasona', 'Gear (Combat)', difficulty=60, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=292, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Glyphs ###
inscription.add("Glyph of the Chosen Glaive", 203762, {194784:5}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Glyph of the Heaved Armament', 203763, {194784:5}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Glyph of the Lunar Chameleon', 211400, {194784:5, 194760:4}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Mark of the Auric Dreamstag', 210735, {194784:5, 204092:1, 194760:4}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Profession Equipment ###
inscription.add("Aclehmist's Brilliant Mixing Rod", 191231, {190456:300, 194864:12, 189143:30}, 'Mellasona', 'Gear (Profession)', difficulty=375, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=282, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Chef's Splendid Rolling Pin", 191232, {194864:12, 189143:30}, 'Mellasona', 'Gear (Profession)', difficulty=375, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=282, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Scribe's Resplendent Quill", 194875, {190456:300, 193053:1, 194864:12, 194756:4}, 'Mellasona', 'Gear (Profession)', difficulty=375, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=282, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Alchemist's Sturdy Mixing Rod", 191234, {194864:1, 194858:1}, 'Mellasona', 'Gear (Profession)', difficulty=80, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=282, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Chef's Smooth Rolling Pin", 191233, {194864:1, 194751:1}, 'Mellasona', 'Gear (Profession)', difficulty=80, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=282, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Scribe's Fastened Quill", 194874, {193053:1, 194864:1, 194858:1}, 'Mellasona', 'Gear (Profession)', difficulty=80, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=282, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)

### Scrolls ###
inscription.add('Illusion Parchment: Aqua Torrent', 194733, {194784:3, 198487:3, 190329:1, 194858:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Illusion Parchment: Arcane Burst', 194735,  {194784:3, 198487:3, 190324:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Illusion Parchment: Chilling Wind', 194736, {194784:3, 198487:3, 190329:1, 194858:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Illusion Parchment: Love Charm', 194732, {194784:3, 198487:3, 190450:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Illusion Parchment: Magma Missile', 194731, {194784:3, 198487:3, 191323:1, 194751:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Illusion Parchment: Shadow Orb', 194738, {194784:3, 198487:3, 190331:1, 194858:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Illusion Parchment: Spell Shield', 194737, {194784:3, 198487:3, 190324:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Scroll of Sales', 198617, {194784:1, 198615:1, 194858:1}, 'Mellasona', 'Other', difficulty=150, insp_percent=18.3, multicraft=13.2, quantity=2, skill=182, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Mysteries ###
inscription.add("Bundle O' Cards: Dragon Isles", 194827, {190324:3, 190327:3, 190316:3, 190321:3, 190329:3, 194784:3}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=19.5, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Fated Fortune Card', 194829, {194784:1, 198487:2, 194751:2}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Fated Fortune Card', 194829, {194784:1, 198487:2, 194852:2}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Fated Fortune Card', 194829, {194784:1, 198487:2, 194858:2}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Contracts ###
inscription.add("Contract: Artisan's Consortium", 198505, {194784:1, 194760:3}, 'Mellasona', 'Other', difficulty=300, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Contract: Dragonscale Expedition', 198508, {194784:1, 194751:1, 194859:1}, 'Mellasona', 'Other', difficulty=300, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Contract: Dream Wardens', 210246, {194784:1, 204460:20, 194760:2}, 'Mellasona', 'Other', difficulty=300, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Contract: Loamm Niffen', 204992, {194784:1, 204460:20, 194760:2}, 'Mellasona', 'Other', difficulty=300, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Contract: Iskaara Tuskarr', 198496, {194784:1, 194858:1, 194859:1}, 'Mellasona', 'Other', difficulty=300, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Contract: Maruuk Centaur', 198502, {194784:1, 194852:1, 194859:1}, 'Mellasona', 'Other', difficulty=300, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Profession Specialization ###
inscription.add('Draconic Treatise on Alchemy', 194697, {190456:10, 193254:1, 194784:5, 190327:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Blacksmithing', 198454, {190456:10, 193254:1, 194784:5, 190321:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Enchanting', 194702, {190456:10, 193254:1, 194784:5, 190329:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Engineering', 198510, {190456:10, 193254:1, 194784:5, 190321:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Herbalism', 194704, {190456:10, 193254:1, 194784:5, 190316:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Inscription', 194699, {190456:10, 193254:1, 194784:5, 190329:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Jewelcrafting', 194703, {190456:10, 193254:1, 194784:5, 190321:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Leatherworking', 194700, {190456:10, 193254:1, 194784:5, 190327:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Mining', 194708, {190456:10, 193254:1, 194784:5, 190316:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Skinning', 201023, {190456:10, 193254:1, 194784:5, 190316:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Draconic Treatise on Tailoring', 194698, {190456:10, 193254:1, 194784:5, 190329:1, 194760:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Dragonriding - Renewed Proto-Drake
inscription.add('Renewed Proto-Drake: Silver and Blue Armor', 197347, {193922:20, 193259:10, 190324:8, 197736:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Renewed Proto-Drake: Steel and Yellow Armor', 197352, {193922:20, 193259:10, 190324:8, 197735:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Renewed Proto-Drake: Bovine Horns', 197377, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Renewed Proto-Drake: Predator Pattern', 197394, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Renewed Proto-Drake: Spined Crest', 197362, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Dragonriding - Windborne Velocidrake ###
inscription.add('Windborne Velocidrake: Silver and Blue Armor', 197578, {193922:20, 193259:10, 190324:8, 197736:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Windborne Velocidrake: Steel and Orange Armor', 197579, {193922:20, 193259:10, 190324:8, 197735:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Windborne Velocidrake: Black Fur', 197597, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Windborne Velocidrake: Spined Head', 197592, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Windborne Velocidrake: Windswept Pattern', 197634, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Dragonriding - Highland Drake ###
inscription.add('Highland Drake: Silver and Blue Armor', 197091, {193922:20, 193259:10, 190324:8, 197736:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Highland Drake: Steel and Yellow Armor', 197096, {193922:20, 193259:10, 190324:8, 197736:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Highland Drake: Black Hair', 197117, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Highland Drake: Spined Neck', 197154, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Dragonriding - Cliffside Wylderdrake ###
inscription.add('Cliffside Wylderdrake: Silver and Blue Armor', 196963, {193922:20, 193259:10, 190324:8, 197736:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Cliffside Wylderdrake: Steel and Yellow Armor', 196968, {193922:20, 193259:10, 190324:8, 197736:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Cliffside Wylderdrake: Conical Head', 196981, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Cliffside Wylderdrake: Red Hair', 196988, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Dragonriding - Winding Slitherdrake ###
inscription.add('Winding Slitherdrake: Blue and Silver Armor', 203300, {193922:20, 193259:10, 190324:8, 197736:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Winding Slitherdrake: Curved Chin Horn', 203314, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Winding Slitherdrake: Hairy Chin', 203311, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Winding Slitherdrake: Small Finned Throat', 203364, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Winding Slitherdrake: White Hair', 203324, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Winding Slitherdrake: Yellow and Silver Armor', 203304, {193922:20, 193259:10, 190324:8, 197735:1, 193254:10}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Dragonriding - Grotto Netherwing Drake ###
inscription.add('Grotto Netherwing Drake: Chin Tendrils', 207761, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Grotto Netherwing Drake: Spiked Jaw', 207773, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, 'Mellasona', 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artisan Curios ###
inscription.add('Arcane Dispelling Rune', 203412, {203403:1}, 'Mellasona', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ###
inscription.add("Draconic Combatant's Emblem", 211608, {190450:4, 194859:1, 194852:10}, None, 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Draconic Combatant's Insignia of Alacrity", 211607, {190450:4, 194859:1, 194751:10}, None, 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Draconic Combatant's Medallion", 211606, {190450:4, 194859:1, 194858:10}, None, 'Gear (Combat)', difficulty=120, insp_percent=33.3, multicraft=np.nan, quantity=1, skill=182, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add("Weathered Explorer's Stave", 194894, {206959:2, 190331:8, 194756:12, 194864:8, 194859:5}, None, 'Gear (Combat)', difficulty=350, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=292, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
inscription.add("Glyph of the Shath'Yar", 207088, {194784:5, 194760:4}, None, 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Illusion Parchment: Whirling Breeze', 194734, {194784:3, 198487:3, 190327:1, 194852:1}, None, 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=5, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Contract: Valdrakken Accord', 198499, {194784:1, 194760:2}, None, 'Other', difficulty=300, insp_percent=38.3, multicraft=5.9, quantity=1, skill=232, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Highland Drake: Spined Head', 197108, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, None, 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
inscription.add('Cliffside Wylderdrake: Triple Head Horns', 196980, {193922:15, 194784:5, 198487:8, 193259:10, 190324:4}, None, 'Dragonriding', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)


### Jewelcrafting

In [14]:
# Last Updates:
# Trees: 90/90 && 200/200 && 210/210 && 130/130
# Gear: 35 Res + 53 Speed + 6 Skill && 53 Insp + 35 Multi + 6 Skill && (110 Multi + 45 Res || 110 Insp + 40 Insp) + 10 Skill

jewelcrafting = Profession('Jewelcrafting')

### Reagents ###
jewelcrafting.add("Dreamtender's Charm", 208748, {208212:1, 210456:1, 192836:1, 192861:2, 192887:1}, 'Nystelil', 'Embellishment', difficulty=340, insp_percent=36.3, multicraft=5.0, quantity=1, skill=227, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Elemental Harmony', 193379, {192852:1, 192858:1, 192861:1, 192862:1, 192868:1, 191495:1}, 'Nystelil', 'Crafting Reagent', difficulty=275, insp_percent=36.3, multicraft=15.0, quantity=2, skill=227, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Illustrious Insight', 191529, {190456:50}, 'Nystelil', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Blotting Sand', 192896, {190320:1, 193922:1, 193370:1}, 'Nystelil', 'Crafting Reagent', difficulty=225, insp_percent=21.3, multicraft=15.0, quantity=2, skill=227, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Pounce', 192899, {190315:2, 193922:1, 193370:2}, 'Nystelil', 'Crafting Reagent', difficulty=275, insp_percent=36.3, multicraft=5.0, quantity=2, skill=227, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Empty Soul Cage', 200938, {192872:3, 190326:2, 190315:2, 190320:2, 190328:2}, 'Nystelil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=15.0, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Draconic Vial', 191476, {190320:1, 192872:5, 200860:5, 193370:1}, 'Nystelil', 'Crafting Reagent', difficulty=300, insp_percent=34.3, multicraft=5.0, quantity=5, skill=237, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Frameless Lens', 192878, {190315:1, 192872:3, 193370:2}, 'Nystelil', 'Crafting Reagent', difficulty=300, insp_percent=34.3, multicraft=5.0, quantity=2, skill=237, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Glossy Stone', 192885, {192880:4, 190315:2, 192851:1}, 'Nystelil', 'Crafting Reagent', difficulty=200, insp_percent=21.3, multicraft=15.0, quantity=2, skill=227, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Shimmering Clasp', 192836, {192833:1, 192842:1}, 'Nystelil', 'Crafting Reagent', difficulty=200, insp_percent=21.3, multicraft=15.0, quantity=1, skill=227, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Rudimentary Gems ### 
jewelcrafting.add("Crafty Queen's Ruby", 192902, {190326:2, 192839:2}, 'Nystelil', 'Item Enhancement', difficulty=60, insp_percent=19.3, multicraft=15.0, quantity=1, skill=207, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Energized Vibrant Emerald', 192908, {190328:2, 192845:2}, 'Nystelil', 'Item Enhancement', difficulty=60, insp_percent=19.3, multicraft=15.0, quantity=1, skill=207, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Sensei's Sundered Onyx", 192912, {190320:2, 192848:2}, 'Nystelil', 'Item Enhancement', difficulty=60, insp_percent=19.3, multicraft=15.0, quantity=1, skill=207, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Zen Mystic Sapphire', 192905, {190315:2, 192842:2}, 'Nystelil', 'Item Enhancement', difficulty=60, insp_percent=19.3, multicraft=15.0, quantity=1, skill=207, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Solid Eternity Amber', 192916, {192851:2}, 'Nystelil', 'Item Enhancement', difficulty=150, insp_percent=19.3, multicraft=15.0, quantity=1, skill=207, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Air Gems ###
jewelcrafting.add('Crafty Alexstraszite', 192919, {190327:2, 190321:1, 190324:1, 192852:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False) 
jewelcrafting.add('Energized Malygite', 192935, {190327:2, 190329:1, 190324:1, 192858:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Forecful Nozdorite', 192973, {190327:1, 192868:1}, 'Nystelil', 'Item Enhancement', difficulty=300, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Keen Neltharite', 192961, {190327:2, 190316:1, 190324:1, 192862:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Quick Ysemerald', 192955, {190327:3, 190324:1, 192861:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Earth Gems ###
jewelcrafting.add('Fractured Neltharite', 192967, {190316:3, 190324:1, 192862:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Keen Ysemerald', 192948, {190316:2, 190327:1, 190324:1, 192861:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Puissant Nozdorite', 192976, {190316:1, 192868:1}, 'Nystelil', 'Item Enhancement', difficulty=300, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Sensei's Alexstraszite", 192922, {190316:2, 190321:1, 190324:1, 192852:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Zen Malygite', 192938, {190316:2, 190329:1, 190324:1, 192858:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Fire Gems ###
jewelcrafting.add('Crafty Ysemerald', 192945, {190321:2, 190327:1, 190324:1, 192861:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Deadly Alexstraszite', 192928, {190321:3, 190324:1, 192852:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Jagged Malygite', 192970, {190321:1, 192868:1}, 'Nystelil', 'Item Enhancement', difficulty=300, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Radiant Malygite', 192932, {190321:2, 190329:1, 190324:1, 192858:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Sensei's Neltharite", 192958, {190321:2, 190316:1, 190324:1, 192862:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Frost Gems ###
jewelcrafting.add('Energized Ysemerald', 192952, {190329:2, 19032:1, 190324:1, 192861:1}, 'Nystelil', 'Item Enhancement', difficulty=325,insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Radiant Alexstraszite', 192925, {190329:2, 190321:1, 190324:1, 192852:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Steady Nozdorite', 192979, {190329:1, 192868:1}, 'Nystelil', 'Item Enhancement', difficulty=300, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Stormy Malygite', 192942, {190329:3, 190324:1, 192858:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Zen Neltharite', 192964, {190329:2, 190316:1, 190324:1, 192862:1}, 'Nystelil', 'Item Enhancement', difficulty=325, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Primalist Gems ###
jewelcrafting.add('Fierce Illimited Diamond', 192985, {190327:1, 190324:1, 192871:1, 192861:1}, 'Nystelil', 'Item Enhancement', difficulty=375, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Inscribed Illimited Diamond', 192982, {190321:1, 190324:1, 192871:1, 192852:1}, 'Nystelil', 'Item Enhancement', difficulty=375, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Resplendent Illimited Diamond', 192991, {190329:1, 190324:1, 192871:1, 192858:1}, 'Nystelil', 'Item Enhancement', difficulty=375, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Skillful Illimited Diamond', 192988, {190316:1, 190324:1, 192871:1, 192862:1}, 'Nystelil', 'Item Enhancement', difficulty=375, insp_percent=34.3, multicraft=5.0, quantity=1, skill=267, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Miscellaneous ###
jewelcrafting.add('Tiered Medallion Setting', 192994, {190324:1, 192836:1, 192851:20, 193370:12}, 'Nystelil', 'Crafting Reagent', difficulty=275, insp_percent=34.8, multicraft=3.2, quantity=1, skill=192, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Unstable Elementium', 204217, {204215:10, 193370:25}, 'Nystelil', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Trinkets ###
jewelcrafting.add('Idol of the Dreamer', 193005, {206959:1, 193921:1, 192861:1, 192871:1, 192885:10}, 'Nystelil', 'Gear (Combat)', difficulty=325, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Idol of the Earth-Warder', 193006, {206959:1, 193920:1, 192862:1, 192871:1, 192885:10}, 'Nystelil', 'Gear (Combat)', difficulty=325, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Idol of the Life-Binder', 193003, {206959:1, 193362:1, 192852:1, 192871:1, 192885:10}, 'Nystelil', 'Gear (Combat)', difficulty=325, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Idol of the Spell-Weaver', 193004, {206959:1, 193919:1, 192858:1, 192871:1, 192885:10}, 'Nystelil', 'Gear (Combat)', difficulty=325, insp_percent=35.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)

### Jewelery ###
jewelcrafting.add('Choker of Shielding', 193002, {206959:1, 192836:2, 192871:1, 192887:1}, 'Nystelil', 'Gear (Combat)', difficulty=315, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Elemental Lariat', 193001, {206959:1, 192836:2, 192871:1, 192887:1}, 'Nystelil', 'Gear (Combat)', difficulty=315, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Ring-Bound Hourglass', 193000, {206959:1, 192836:2, 192871:1, 192887:1, 193370:3}, 'Nystelil', 'Gear (Combat)', difficulty=315, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Signet of Titanic Insight', 192999, {206959:1, 192836:2, 192887:1, 192861:1}, 'Nystelil', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Torc of Passed Time', 201759, {206959:1, 192836:2, 192887:1, 192858:1}, 'Nystelil', 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add("Obsidian Combatant's Jeweled Amulet", 204095, {190451:3, 192836:1, 192842:3}, 'Nystelil', 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Obsidian Combatant's Jeweled Signet", 204094, {190451:3, 192836:1, 192842:3}, 'Nystelil', 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Verdant Combatant's Jeweled Amulet", 208238, {190451:8, 192836:1, 192842:5}, 'Nystelil', 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Verdant Combatant's Jeweled Signet", 208237, {190451:8, 192836:1, 192842:5}, 'Nystelil', 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Crimson Combatant's Jeweled Amulet", 192998, {190451:2, 192836:1, 192842:2}, 'Nystelil', 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Crimson Combatant's Jeweled Signet", 192997, {190451:2, 192836:1, 192842:2}, 'Nystelil', 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Band of New Beginnings', 194082, {190320:5, 192836:1, 192851:2}, 'Nystelil', 'Gear (Combat)', difficulty=60, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
jewelcrafting.add('Pendant of Impending Perils', 192996, {192836:1, 192848:1}, 'Nystelil', 'Gear (Combat)', difficulty=40, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Statues & Carvings ###
jewelcrafting.add("Djaradin's \"Pinata\"", 193021, {190450:1, 192885:9, 192848:6, 193370:2, 192836:3}, 'Nystelil', 'Consumable', difficulty=375, insp_percent=34.8, multicraft=6.8, quantity=3, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Figure of the Gathering Storm', 205173,  {204463:1, 190450:2, 201406:1, 192885:30, 193370:15}, 'Nystelil', 'Consumable', difficulty=330, insp_percent=33.8, multicraft=6.8, quantity=1, skill=242, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Statuette of Foreseen Power', 205170, {204463:1, 190450:2, 192885:30, 192858:3, 193370:15}, 'Nystelil', 'Consumable', difficulty=330, insp_percent=33.8, multicraft=6.8, quantity=1, skill=242, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Narcissist's Sculpture", 193009, {190321:1, 192885:2, 192845:1, 193370:1, 192836:1}, 'Nystelil', 'Consumable', difficulty=300, insp_percent=34.8, multicraft=6.8, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Kalu'ak Figurine", 194725, {194730:1, 192885:1, 193370:1}, 'Nystelil', 'Consumable', difficulty=250, insp_percent=19.8, multicraft=16.8, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Statue of Tyr's Herald", 193017, {201406:1, 192885:2, 192842:1, 193370:1, 192836:1}, 'Nystelil', 'Consumable', difficulty=300, insp_percent=34.8, multicraft=6.8, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False) 
jewelcrafting.add('Revitalizing Red Carving', 193013, {192885:2, 192839:1, 193370:1, 192836:1}, 'Nystelil', 'Consumable', difficulty=250, insp_percent=19.8, multicraft=16.8, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Battle Pets ###
jewelcrafting.add('Jeweled Onyx Whelpling', 193889, {200867:1, 192885:50, 192862:2, 192887:2, 192871:1}, 'Nystelil', 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Jeweled Sapphire Whelpling', 193886, {200866:1, 192885:50, 192858:2, 192887:2, 192871:1}, 'Nystelil', 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Novelties ###
jewelcrafting.add('Convergent Prism', 193033, {201406:1, 192871:1, 193031:1, 192887:1}, 'Nystelil', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Jeweled Offering', 193032,  {193053:75, 201405:2, 192852:1, 193370:15, 193931:10}, 'Nystelil', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('B.B.F. Fist', 205045, {204463:1, 193920:1, 192885:200, 193370:150, 192868:15}, 'Nystelil', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Projection Prism', 193031, {192872:7, 192878:1, 193370:1}, 'Nystelil', 'Crafting Reagent', difficulty=225, insp_percent=19.3, multicraft=15.0, quantity='1-2', skill=237, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Split-Lens Specs', 194748, {192862:1, 192878:1, 192887:1}, 'Nystelil', 'Cosmetic', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Profession Equipment ###
jewelcrafting.add('Alexstraszite Loupes', 193041, {190456:225, 194124:2, 192878:2, 192836:5, 192852:2}, 'Nystelil', 'Gear (Profession)', difficulty=325, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False) 
jewelcrafting.add('Fine-Print Trifocals', 193039, {190456:225, 194124:2, 192878:2, 192836:5, 192868:2}, 'Nystelil', 'Gear (Profession)', difficulty=325, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Magnificent Margin Magnifier', 193040, {190456:225, 192878:2, 192836:2, 192868:4}, 'Nystelil', 'Gear (Profession)', difficulty=325, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Resonant Focus', 193042, {190456:225, 200113:2, 194124:2, 192836:2}, 'Nystelil', 'Gear (Profession)', difficulty=325, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Bold-Print Bifocals', 193035, {192878:2, 190395:2}, 'Nystelil', 'Gear (Profession)', difficulty=80, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Chromatic Focus', 193038, {194123:3, 192836:1}, 'Nystelil', 'Gear (Profession)', difficulty=80, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Left-Handed Magnifying Glass', 193036, {192878:1, 189143:3}, 'Nystelil', 'Gear (Profession)', difficulty=80, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Sundered Onyx Loupe', 193037, {192878:2, 190395:2, 192848:1}, 'Nystelil', 'Gear (Profession)', difficulty=80, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=227, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Extravagant Glasswares ###
jewelcrafting.add("Jeweled Dragon's Heart", 192893, {192839:1, 192842:1, 192845:1, 192848:1, 192851:2, 193370:3}, 'Nystelil', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Dreamer's Vision", 202055, {192872:3, 192845:2, 193370:1}, 'Nystelil', 'Other', difficulty=250, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=237, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Earthwarden's Prize", 202057, {192872:3, 192848:2, 193370:1}, 'Nystelil', 'Other', difficulty=250, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=237, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Keeper's Glory", 202056, {192872:3, 192842:2, 193370:1}, 'Nystelil', 'Other', difficulty=250, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=237, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Queen's Gift", 202054, {192872:3, 192839:2, 193370:1}, 'Nystelil', 'Other', difficulty=250, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=237, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Timewatcher's Patience", 202058, {192872:3, 192851:4, 193370:1}, 'Nystelil', 'Other', difficulty=250, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=237, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artison Curios ###
jewelcrafting.add("Crystal Tuning Fork", 203413, {203404:1}, 'Nystelil', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ###
jewelcrafting.add("Draconic Combatant's Jeweled Amulet", 211537, {190451:8, 192836:1, 192842:5}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add("Draconic Combatant's Jeweled Signet", 211536, {190451:8, 192836:1, 192842:5}, None, 'Gear (Combat)', difficulty=120, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Jeweled Amber Whelpling', 193885, {200863:1, 192885:50, 192868:2, 192887:2, 192871:1}, None, 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Jeweled Emerald Whelpling', 193888, {200865:1, 192885:50, 192861:2, 192887:2, 192871:1}, None, 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('Jeweled Ruby Whelpling', 193887, {200864:1, 192885:50, 192852:2, 192887:2, 192871:1}, None, 'Pet', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
jewelcrafting.add('"Rhinestone" Sunglasses', 193371, {192861:2, 192878:2, 192871:1, 192887:1}, None, 'Cosmetic', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Leatherworking

In [15]:
### Shalzin ###
leatherworking1 = Profession('Leatherworking')

leatherworking1.add("Flame-Touched Chain", 193425, {20440: 1, 193251: 4, 193244: 14, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Flame-Touched Chainmail", 193422, {206959: 1, 193259: 4, 193228: 15, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Flame-Touched Cuffs", 193428, {206959: 1, 193256: 4, 193244: 10, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Flame-Touched Handguards", 193427, {206959: 1, 193254: 4, 193244: 13, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Flame-Touched Helmet", 193423, {206959: 1, 193253: 4, 193228: 15, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Flame-Touched Legguards", 193426, {206959: 1, 193253: 4, 193228: 15, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Flame-Touched Spaulders", 193424, {206959: 1, 193256: 4, 193244: 13, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Flame-Touched Treads", 193421, {206959: 1, 193251: 4, 193244: 13, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Toughened Coif", 201946, {193215: 16, 193224: 2}, "Shalzin", 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Toughened Legguards", 201949, {193215: 16, 193224: 2}, "Shalzin", 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Toughened Spikes", 201947, {193215: 16, 193224: 2}, "Shalzin", 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Toughened Grips", 201948, {193215: 16, 193224: 2}, "Shalzin", 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Toughened Chainbelt", 201950, {193215: 16, 193224: 2}, "Shalzin", 'Gear (Combat)', difficulty=60, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Scale Vest", 193394, {193215: 25, 193208: 15}, "Shalzin", 'Gear (Combat)', difficulty=40, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Scale Boots", 193392, {193215: 20, 193208: 10}, "Shalzin", 'Gear (Combat)', difficulty=40, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Trailblazer's Scale Bracers", 193393, {193215: 15, 193208: 5}, "Shalzin", 'Gear (Combat)', difficulty=40, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Alchemist's Hat", 193487, {193224: 2, 193208: 10}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Smithing Apron", 193612, {193218: 2, 193215: 10}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Jeweler's Cover", 193615, {193218: 2, 193208: 10}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Protective Gloves", 193485, {193224: 2, 193215: 10}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Durable Pack", 193480, {193208: 20, 193218: 1}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Floral Basket", 193479, {193218: 2, 193208: 5, 191462: 5}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Skinner's Cap", 193482, {193208: 30, 193224: 1}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Resilient Smock", 193486, {193208: 20, 193215: 10}, "Shalzin", 'Gear (Profession)', difficulty=80, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Bonewrought Crossbow", 193383, {193255: 2, 194864: 1, 193208: 20}, "Shalzin", 'Gear (Combat)', difficulty=60, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Scale Rein Grips", 193465, {206959: 1, 193252: 12, 193244: 18, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=415, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Bow of the Dragon Hunters", 193449, {206959: 2, 190327: 6, 201404: 2, 193231: 10, 194864: 2}, "Shalzin", 'Gear (Combat)', difficulty=280, insp_percent=36.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Infurious Boots of Reprieve", 193461, {206959: 1, 193360: 1, 193238: 16, 193215: 100}, "Shalzin", 'Gear (Combat)', difficulty=415, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Infurious Chainhelm Protector", 193462, {206959: 1, 193360: 1, 193250: 20, 193215: 100}, "Shalzin", 'Gear (Combat)', difficulty=415, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Obsidian Combatant's Adamant Chainmail", 204105, {193253: 4, 193250: 2, 193215: 20}, "Shalzin", 'Gear (Combat)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Obsidian Combatant's Adamant Cuffs", 204111, {193251: 4, 193250: 2, 193215: 10}, "Shalzin", 'Gear (Combat)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Obsidian Combatant's Adamant Epaulettes", 204107, {193258: 4, 193250: 2, 193215: 16}, "Shalzin", 'Gear (Combat)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Obsidian Combatant's Adamant Girdle", 204108, {193252: 4, 193250: 2, 193215: 10}, "Shalzin", 'Gear (Combat)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Obsidian Combatant's Adamant Leggings", 204109, {193254: 4, 193250: 2, 193215: 20}, "Shalzin", 'Gear (Combat)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Obsidian Combatant's Adamant Treads", 204104, {193251: 4, 193250: 2, 193215: 16}, "Shalzin", 'Gear (Combat)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Acidic Hailstone Treads", 193466, {206959: 1, 193254: 12, 193247: 18, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=415, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Venom-Steeped Stompers", 193460, {206959: 1, 193253: 6, 193228: 18, 193215: 150}, "Shalzin", 'Gear (Combat)', difficulty=415, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Witherrot Tome", 193496, {206959: 1, 190331: 15, 201404: 2, 193228: 14, 193208: 100}, "Shalzin", 'Gear (Combat)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=247, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Earthshine Scales", 193244, {190316: 1, 190329: 1, 193224: 2}, "Shalzin", 'Crafting Reagent', difficulty=375, insp_percent=36.3, multicraft=2.9, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Frostbite Scales", 193247, {190329: 1, 190331: 1, 193224: 2}, "Shalzin", 'Crafting Reagent', difficulty=375, insp_percent=36.3, multicraft=2.9, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Mireslush Hide", 193231, {190316: 1, 190329: 1, 193218: 2}, "Shalzin", 'Crafting Reagent', difficulty=375, insp_percent=36.3, multicraft=2.9, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Stonecrust Hide", 193228, {190331: 1, 190316: 1, 193218: 2}, "Shalzin", 'Crafting Reagent', difficulty=375, insp_percent=36.3, multicraft=2.9, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Fang Adornments", 193554, {193254: 2, 193255: 2, 201400: 1, 201403: 1, 193244: 3, 193208: 60}, "Shalzin", 'Embellishment', difficulty=350, insp_percent=34.3, multicraft=2.9, quantity=1, skill=247, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Toxified Armor Patch", 193555, {190331: 8, 193228: 2, 193215: 60}, "Shalzin", 'Embellishment', difficulty=350, insp_percent=34.3, multicraft=2.9, quantity=1, skill=247, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Illustrious Insight", 191529, {190456: 50}, "Shalzin", 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Reinforced Armor Kit", 193567, {193253: 2, 193218: 1, 193208: 50}, "Shalzin", 'Consumable', difficulty=150, insp_percent=34.3, multicraft=8.4, quantity=1, skill=197, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Feral Hide Drums", 193470, {193208: 100, 193215: 100}, "Shalzin", 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=8.4, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Artisan's Sign", 197719, {198615: 4, 193254: 5, 193218: 2, 194864: 4}, "Shalzin", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Gnoll Tent", 193476, {201399: 1, 193218: 20, 193224: 15, 193208: 200}, "Shalzin", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Adaptive Dracothyst Armguards", 204704, {206959: 1, 204463: 1, 205413: 1, 193244: 10, 193215: 150}, None, 'Gear (Combat)', difficulty=330, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Expert Alchemist's Hat", 193493, {190456: 225, 190327: 12, 193247: 8, 193208: 80}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Expert Skinner's Cap", 193490, {190456: 225, 193256: 4, 201405: 2, 193244: 10, 193208: 80}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Flameproof Apron", 193613, {190456: 225, 193259: 10, 193228: 8, 193215: 80}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Lavish Floral Pack", 193488, {190456: 225, 193251: 15, 193231: 10, 193208: 60, 191498: 5}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Masterwork Smock", 193492, {190456: 225, 193251: 15, 193231: 10, 193208: 80}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Reinforced Pack", 193489, {190456: 225, 193255: 15, 193244: 10, 193208: 80}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Resplendent Cover", 193616, {190456: 225, 193252: 20, 193228: 10, 193208: 80}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Shockproof Gloves", 193491, {190456: 225, 193254: 15, 193244: 10, 193215: 80}, None, 'Gear (Profession)', difficulty=275, insp_percent=34.3, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Ancestor's Dew Drippers", 193459, {206959: 1, 193251: 10, 193231: 18, 193215: 150}, None, 'Gear (Profession)', difficulty=415, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Wind Spirit's Lasso", 193463, {206959: 1, 190327: 10, 193231: 16, 193215: 150}, None, 'Gear (Profession)', difficulty=415, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Allied Legguards of Sansok Khan", 193464, {206959: 1, 193360: 1, 193244: 20, 193215: 150}, None, 'Gear (Profession)', difficulty=435, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking1.add("Obsidian Combatant's Adamant Cowl", 204106, {193256: 4, 193250: 2, 193215: 20}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Obsidian Combatant's Adamant Gauntlets", 204110, {193252: 4, 193250: 2, 193215: 16}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Chainmail", 193440, {193253: 2, 193250: 1, 193215: 15}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Cowl", 193441, {193256: 2, 193250: 1, 193215: 15}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Cuffs", 193446, {193251: 2, 193250: 1, 193215: 10}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Epaulettes", 193442, {193258: 2, 193250: 1, 193215: 12}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Gauntlets", 193445, {193252: 2, 193250: 1, 193215: 12}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Girdle", 193443, {193252: 2, 193250: 1, 193215: 10}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Leggings", 193444, {193254: 2, 193250: 1, 193215: 15}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Crimson Combatant's Adamant Treads", 193439, {193251: 2, 193250: 1, 193215: 12}, None, 'Gear (Profession)', difficulty=120, insp_percent=38.8, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Finished Prototype Explorer's Barding", 197735, {194542: 1, 201405: 1, 193224: 6, 193208: 80}, None, 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Finished Prototype Regal Barding", 197736, {194541: 1, 201405: 1, 193224: 6, 193208: 80}, None, 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Infurious Hide", 193238, {190450: 2, 193218: 2}, None, 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=2.9, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Infurious Scales", 193250, {190450: 2, 193224: 2}, None, 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=2.9, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Shadowflame-Tempered Armor Patch", 204710, {193254: 2, 193255: 2, 204464: 1, 190321: 2, 193244: 2, 193208: 150}, None, 'Embellishment', difficulty=360, insp_percent=34.3, multicraft=2.9, quantity=1, skill=247, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Fierce Armor Kit", 193565, {190316: 10, 193228: 4, 193208: 60}, None, 'Consumable', difficulty=375, insp_percent=36.3, multicraft=8.4, quantity=1, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Frosted Armor Kit", 193564, {190329: 10, 193247: 4, 193215: 60}, None, 'Consumable', difficulty=375, insp_percent=36.3, multicraft=8.4, quantity=1, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Lambent Armor Kit", 204702, {204460: 150, 193228: 4, 193208: 60}, None, 'Consumable', difficulty=300, insp_percent=34.3, multicraft=8.4, quantity=1, skill=197, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Feral Hide Drums", 193470, {193218: 1}, None, 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=8.4, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Feral Hide Drums", 193470, {193224: 1}, None, 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=8.4, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Tuskarr Beanbag", 193478, {193053: 100, 193218: 30, 193208: 200}, None, 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking1.add("Reinforced Pristine Leather", 203414, {203405: 1}, None, 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=2.9, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

In [16]:
# Last Updates:
# Trees: 140/140 && 250/250 && 250/250 && 110/110
# Gear: 53 Insp + 35 Multi + 6 Skill && 44 Res + 44 Speed + 6 Skill && (110 Insp + 40 Insp || 110 Multi + 45 Res) + 10 Skill

### Braevele ###
leatherworking2 = Profession('Leatherworking')

### Leather Armor ###
leatherworking2.add("Life-Bound Belt", 193407, {206959: 1, 193255: 4, 193244: 13, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Life-Bound Bindings", 193419, {206959: 1, 193256: 3, 193244: 10, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Life-Bound Boots", 193398, {206959: 1, 193251: 4, 193247: 13, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Life-Bound Cap", 193400, {206959: 1, 193256: 4, 193231: 15, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Life-Bound Chestpiece", 193399, {206959: 1, 193259: 4, 193231: 15, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Life-Bound Gloves", 193418, {206959: 1, 193252: 4, 193247: 13, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Life-Bound Shoulderpads", 193406, {206959: 1, 193258: 4, 193228: 13, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Life-Bound Trousers", 193408, {206959: 1, 193253: 4, 193228: 15, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Practiced Cowl", 201941, {193208: 16, 193218: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Practiced Leggings", 201944, {193208: 16, 193218: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Practiced Shoulderpads", 201942, {193208: 16, 193218: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Practiced Gloves", 201943, {193208: 16, 193218: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Practiced Belt", 201945, {193208: 16, 193218: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Leather Tunic", 193390, {193208: 25, 193215: 15}, "Braevele", 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Leather Boots", 193386, {193208: 20, 193215: 10}, "Braevele", 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Pioneer's Leather Wristguards", 193388, {193208: 15, 193215: 5}, "Braevele", 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Mail Armor ###
leatherworking2.add("Adaptive Dracothyst Armguards", 204704, {206959: 1, 204463: 1, 205413: 1, 193244: 10, 193215: 150}, 'Braevele', 'Gear (Combat)', difficulty=330, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Chain", 193425, {20440: 1, 193251: 4, 193244: 14, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Chainmail", 193422, {206959: 1, 193259: 4, 193228: 15, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Cuffs", 193428, {206959: 1, 193256: 4, 193244: 10, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Handguards", 193427, {206959: 1, 193254: 4, 193244: 13, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Helmet", 193423, {206959: 1, 193253: 4, 193228: 15, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Legguards", 193426, {206959: 1, 193253: 4, 193228: 15, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Spaulders", 193424, {206959: 1, 193256: 4, 193244: 13, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flame-Touched Treads", 193421, {206959: 1, 193251: 4, 193244: 13, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Toughened Coif", 201946, {193215: 16, 193224: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Toughened Legguards", 201949, {193215: 16, 193224: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Toughened Spikes", 201947, {193215: 16, 193224: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Toughened Grips", 201948, {193215: 16, 193224: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Toughened Chainbelt", 201950, {193215: 16, 193224: 2}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Scale Vest", 193394, {193215: 25, 193208: 15}, "Braevele", 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Scale Boots", 193392, {193215: 20, 193208: 10}, "Braevele", 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Trailblazer's Scale Bracers", 193393, {193215: 15, 193208: 5}, "Braevele", 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Profession Equipment ###
leatherworking2.add("Expert Alchemist's Hat", 193493, {190456: 225, 190327: 12, 193247: 8, 193208: 80}, 'Braevele', 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Expert Skinner's Cap", 193490, {190456: 225, 193256: 4, 201405: 2, 193244: 10, 193208: 80}, "Braevele", 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Flameproof Apron", 193613, {190456: 225, 193259: 10, 193228: 8, 193215: 80}, 'Braevele', 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Lavish Floral Pack", 193488, {190456: 225, 193251: 15, 193231: 10, 193208: 60, 191498: 5}, "Braevele", 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Masterwork Smock", 193492, {190456: 225, 193251: 15, 193231: 10, 193208: 80}, 'Braevele', 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Reinforced Pack", 193489, {190456: 225, 193255: 15, 193244: 10, 193208: 80}, 'Braevele', 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Resplendent Cover", 193616, {190456: 225, 193252: 20, 193228: 10, 193208: 80}, "Braevele", 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Shockproof Gloves", 193491, {190456: 225, 193254: 15, 193244: 10, 193215: 80}, "Braevele", 'Gear (Profession)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Alchemist's Hat", 193487, {193224: 2, 193208: 10}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Smithing Apron", 193612, {193218: 2, 193215: 10}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Jeweler's Cover", 193615, {193218: 2, 193208: 10}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Protective Gloves", 193485, {193224: 2, 193215: 10}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Durable Pack", 193480, {193208: 20, 193218: 1}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Floral Basket", 193479, {193218: 2, 193208: 5, 191462: 5}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Skinner's Cap", 193482, {193208: 30, 193224: 1}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Resilient Smock", 193486, {193208: 20, 193215: 10}, "Braevele", 'Gear (Profession)', difficulty=80, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Weapons ###
leatherworking2.add("Bonewrought Crossbow", 193383, {193255: 2, 194864: 1, 193208: 20}, "Braevele", 'Gear (Combat)', difficulty=60, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=197, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Elemental Patterns ###
leatherworking2.add("Ancestor's Dew Drippers", 193459, {206959: 1, 193251: 10, 193231: 18, 193215: 150}, 'Braevele', 'Gear (Profession)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Flaring Cowl", 193494, {206959: 1, 193362: 1, 190321: 10, 193244: 16, 193208: 100}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Old Spirit's Wristwraps", 193454, {206959: 1, 193254: 10, 193231: 16, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Scale Rein Grips", 193465, {206959: 1, 193252: 12, 193244: 18, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Snowball Makers", 193458, {206959: 1, 193251: 10, 193247: 20, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("String of Spiritual Knick-Knacks", 193457, {206959: 1, 193252: 10, 193231: 20, 193208: 150}, 'Braevele', 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Wind Spirit's Lasso", 193463, {206959: 1, 190327: 10, 193231: 16, 193215: 150}, 'Braevele', 'Gear (Profession)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)

### Beastial Patterns ###
leatherworking2.add("Allied Heartwarming Fur Coat", 193453, {206959: 1, 193258: 10, 201405: 3, 193247: 20, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Allied Legguards of Sansok Khan", 193464, {206959: 1, 193360: 1, 193244: 20, 193215: 150}, 'Braevele', 'Gear (Profession)', difficulty=435, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Bow of the Dragon Hunters", 193449, {206959: 2, 190327: 6, 201404: 2, 193231: 10, 194864: 2}, "Braevele", 'Gear (Combat)', difficulty=280, insp_percent=36.8, multicraft=np.nan, quantity=1, skill=272, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Infurious Boots of Reprieve", 193461, {206959: 1, 193360: 1, 193238: 16, 193215: 100}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Infurious Chainhelm Protector", 193462, {206959: 1, 193360: 1, 193250: 20, 193215: 100}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Infurious Footwraps of Indemnity", 193455, {206959: 1, 193251: 8, 193250: 16, 193208: 100}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Infurious Spirit's Hood", 193456, {206959: 1, 193258: 8, 193238: 20, 193208: 100}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)

### Beastial - Obsidian Combatant ###
leatherworking2.add("Obsidian Combatant's Adamant Chainmail", 204105, {193253: 4, 193250: 2, 193215: 20}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Adamant Cowl", 204106, {193256: 4, 193250: 2, 193215: 20}, "Braevele", 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Adamant Cuffs", 204111, {193251: 4, 193250: 2, 193215: 10}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Adamant Epaulettes", 204107, {193258: 4, 193250: 2, 193215: 16}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Adamant Gauntlets", 204110, {193252: 4, 193250: 2, 193215: 16}, "Braevele", 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Adamant Girdle", 204108, {193252: 4, 193250: 2, 193215: 10}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Adamant Leggings", 204109, {193254: 4, 193250: 2, 193215: 20}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Adamant Treads", 204104, {193251: 4, 193250: 2, 193215: 16}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Belt", 204100, {193252: 4, 193238: 2, 193208: 10}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Boots", 204096, {193251: 4, 193238: 2, 193208: 16}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Chestpiece", 204097, {193253: 4, 193238: 2, 193208: 20}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Gloves", 204102, {193252: 4, 193238: 2, 193208: 16}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Mask", 204098, {193256: 4, 193238: 2, 193208: 20}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Shoulderpads", 204099, {193258: 4, 193238: 2, 193208: 16}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Trousers", 204101, {193254: 4, 193238: 2, 193208: 20}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Obsidian Combatant's Resilient Wristwraps", 204103, {193251: 4, 193238: 2, 193208: 10}, "Braevele", 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)

### Beastial - Crimson Combatant ###
leatherworking2.add("Crimson Combatant's Adamant Chainmail", 193440, {193253: 2, 193250: 1, 193215: 15}, "Braevele", 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Adamant Cowl", 193441, {193256: 2, 193250: 1, 193215: 15}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Adamant Cuffs", 193446, {193251: 2, 193250: 1, 193215: 10}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Adamant Epaulettes", 193442, {193258: 2, 193250: 1, 193215: 12}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Adamant Gauntlets", 193445, {193252: 2, 193250: 1, 193215: 12}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Adamant Girdle", 193443, {193252: 2, 193250: 1, 193215: 10}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Adamant Leggings", 193444, {193254: 2, 193250: 1, 193215: 15}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Adamant Treads", 193439, {193251: 2, 193250: 1, 193215: 12}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Belt", 193434, {193252: 2, 193238: 1, 193208: 10}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Boots", 193430, {193251: 2, 19323: 1, 193208: 12}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Chestpiece", 193431, {193253: 2, 193238: 1, 193208: 15}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Gloves", 193436, {193252: 2, 193238: 1, 193208: 12}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Mask", 193432, {193256: 2, 193238: 1, 193208: 15}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Shoulderpads", 193433, {193258: 2, 193238: 1, 193208: 12}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Trousers", 193435, {193254: 2, 193238: 1, 193208: 15}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Crimson Combatant's Resilient Wristwraps", 193437, {193251: 2, 193238: 1, 193208: 10}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)

### Beastial - Verdant Combatant ###
leatherworking2.add("Verdant Combatant's Adamant Chainmail", 208248, {193253: 4, 193250: 4, 193215: 25}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Adamant Cowl", 208249, {193256: 4, 193250: 4, 193215: 25}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Adamant Cuffs", 208254, {193251: 4, 193250: 4, 193215: 15}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Adamant Epaulettes", 208250, {193258: 4, 193250: 4, 193215: 20}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Adamant Gauntlets", 208253, {193252: 4, 193250: 4, 193215: 20}, 'Braevele', 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Adamant Girdle", 208251, {193252: 4, 193250: 4, 193215: 15}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Adamant Leggings", 208252, {193254: 4, 193250: 4, 193215: 25}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Adamant Treads", 208247, {193251: 4, 193250: 4, 193215: 20}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Belt", 208243, {193252: 4, 193238: 4, 193208: 15}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Boots", 208239, {193251: 4, 193238: 4, 193208: 20}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Chestpiece", 208240, {193253: 4, 193238: 4, 193208: 25}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Gloves", 208245, {193252: 4, 193238: 4, 193208: 20}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Mask", 208241, {193256: 4, 193238: 4, 193208: 25}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Shoulderpads", 208242, {193258: 4, 193238: 4, 193208: 20}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Trousers", 208244, {193254: 4, 193238: 4, 193208: 25}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Verdant Combatant's Resilient Wristwraps", 208246, {193251: 4, 193238: 4, 193208: 15}, 'Braevele', 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)

### Decayed Patterns ###
leatherworking2.add("Acidic Hailstone Treads", 193466, {206959: 1, 193254: 12, 193247: 18, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Slimy Expulsion Boots", 193451, {206959: 1, 193252: 10, 193228: 18, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Spore Colony Shoulderguards", 204706, {206959: 1, 204460: 150, 193228: 18, 193208: 200}, 'Braevele', 'Gear (Combat)', difficulty=430, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Toxic Thorn Footwraps", 193452, {206959: 1, 193253: 6, 193247: 18, 193208: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Venom-Steeped Stompers", 193460, {206959: 1, 193253: 6, 193228: 18, 193215: 150}, "Braevele", 'Gear (Combat)', difficulty=415, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
leatherworking2.add("Witherrot Tome", 193496, {206959: 1, 190331: 15, 201404: 2, 193228: 14, 193208: 100}, "Braevele", 'Gear (Combat)', difficulty=275, insp_percent=34.8, multicraft=np.nan, quantity=1, skill=247, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True)

### Reagents ###
leatherworking2.add("Finished Prototype Explorer's Barding", 197735, {194542: 1, 201405: 1, 193224: 6, 193208: 80}, 'Braevele', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Finished Prototype Regal Barding", 197736, {194541: 1, 201405: 1, 193224: 6, 193208: 80}, 'Braevele', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Earthshine Scales", 193244, {190316: 1, 190329: 1, 193224: 2}, "Braevele", 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=3.2, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Frostbite Scales", 193247, {190329: 1, 190331: 1, 193224: 2}, "Braevele", 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=3.2, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Infurious Hide", 193238, {190450: 2, 193218: 2}, "Braevele", 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=3.2, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Infurious Scales", 193250, {190450: 2, 193224: 2}, "Braevele", 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=3.2, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Mireslush Hide", 193231, {190316: 1, 190329: 1, 193218: 2}, "Braevele", 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=3.2, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Stonecrust Hide", 193228, {190331: 1, 190316: 1, 193218: 2}, "Braevele", 'Crafting Reagent', difficulty=375, insp_percent=36.8, multicraft=3.2, quantity=2, skill=272, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Optional Reagents ###
leatherworking2.add("Fang Adornments", 193554, {193254: 2, 193255: 2, 201400: 1, 201403: 1, 193244: 3, 193208: 60}, "Braevele", 'Embellishment', difficulty=350, insp_percent=34.8, multicraft=3.2, quantity=1, skill=247, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Shadowflame-Tempered Armor Patch", 204710, {193254: 2, 193255: 2, 204464: 1, 190321: 2, 193244: 2, 193208: 150}, "Braevele", 'Embellishment', difficulty=360, insp_percent=34.8, multicraft=3.2, quantity=1, skill=247, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Toxified Armor Patch", 193555, {190331: 8, 193228: 2, 193215: 60}, "Braevele", 'Embellishment', difficulty=350, insp_percent=34.8, multicraft=3.2, quantity=1, skill=247, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Finishing Reagents ###
leatherworking2.add("Illustrious Insight", 191529, {190456: 50}, "Braevele", 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Armor Kits ###
leatherworking2.add("Fierce Armor Kit", 193565, {190316: 10, 193228: 4, 193208: 60}, 'Braevele', 'Consumable', difficulty=375, insp_percent=36.8, multicraft=8.6, quantity=1, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Frosted Armor Kit", 193564, {190329: 10, 193247: 4, 193215: 60}, 'Braevele', 'Consumable', difficulty=375, insp_percent=36.8, multicraft=8.6, quantity=1, skill=272, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Lambent Armor Kit", 204702, {204460: 150, 193228: 4, 193208: 60}, 'Braevele', 'Consumable', difficulty=300, insp_percent=34.8, multicraft=8.6, quantity=1, skill=197, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Reinforced Armor Kit", 193567, {193253: 2, 193218: 1, 193208: 50}, "Braevele", 'Consumable', difficulty=150, insp_percent=19.8, multicraft=18.6, quantity=1, skill=197, rarity='common', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Drums ###
leatherworking2.add("Feral Hide Drums", 193470, {193208: 100, 193215: 100}, "Braevele", 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=18.6, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Feral Hide Drums", 193470, {193218: 1}, 'Braevele', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=18.6, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Feral Hide Drums", 193470, {193224: 1}, 'Braevele', 'Consumable', difficulty=np.nan, insp_percent=np.nan, multicraft=18.6, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Toys ###
leatherworking2.add("Artisan's Sign", 197719, {198615: 4, 193254: 5, 193218: 2, 194864: 4}, "Braevele", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Gnoll Tent", 193476, {201399: 1, 193218: 20, 193224: 15, 193208: 200}, "Braevele", 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Tuskarr Beanbag", 193478, {193053: 100, 193218: 30, 193208: 200}, 'Braevele', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artisan Curios ###
leatherworking2.add("Reinforced Pristine Leather", 203414, {203405: 1}, "Braevele", 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=13.2, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Not Learned ###
leatherworking2.add("Draconic Combatant's Adamant Chainmail", 211547, {193253: 4, 193250: 4, 193215: 25}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Adamant Cowl", 211548, {193256: 4, 193250: 4, 193215: 25}, None, 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Adamant Cuffs", 211553, {193251: 4, 193250: 4, 193215: 15}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Adamant Epaulettes", 211549, {193258: 4, 193250: 4, 193215: 20}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Adamant Gauntlets", 211552, {193252: 4, 193250: 4, 193215: 20}, None, 'Gear (Profession)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Adamant Girdle", 211550, {193252: 4, 193250: 4, 193215: 15}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Adamant Leggings", 211551, {193254: 4, 193250: 4, 193215: 25}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Adamant Treads", 211546, {193251: 4, 193250: 4, 193215: 20}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Belt", 211542, {193252: 4, 193238: 4, 193208: 15}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Boots", 211538, {193251: 4, 193238: 4, 193208: 20}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Chestpiece", 211539, {193253: 4, 193238: 4, 193208: 25}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Gloves", 211544, {193252: 4, 193238: 4, 193208: 20}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Mask", 211540, {193256: 4, 193238: 4, 193208: 25}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Shoulderpads", 211541, {193258: 4, 193238: 4, 193208: 20}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Trousers", 211543, {193254: 4, 193238: 4, 193208: 25}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add("Draconic Combatant's Resilient Wristwraps", 211545, {193251: 4, 193238: 4, 193208: 15}, None, 'Gear (Combat)', difficulty=120, insp_percent=39.3, multicraft=np.nan, quantity=1, skill=347, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
leatherworking2.add('Verdant Conduit', 208189, {208212:1, 210456:1, 193231:1, 194864:1}, None, 'Embellishment', difficulty=360, insp_percent=34.8, multicraft=3.2, quantity=1, skill=247, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)


In [17]:
leatherworking_temp = pd.concat((leatherworking1.get_table(), leatherworking2.get_table()), ignore_index=True)
leatherworking_temp['reagents_str'] = leatherworking_temp['reagents'].apply(str)

drop_rows = list()
leatherworking_temp = leatherworking_temp.sort_values(by=['id', 'reagents_str', 'character'])
leatherworking_temp = leatherworking_temp.reset_index(drop=True)
for index, row in leatherworking_temp.iterrows():
    if index == 0:
        prev_row = row
        continue

    if (prev_row['id'] == row['id'] and prev_row['reagents_str']==row['reagents_str'] and 
        (row['character'] is pd.NA or row['character'] is None)):
        drop_rows.append(index)
    prev_row = row
    
leatherworking_temp = leatherworking_temp.drop(drop_rows, axis=0)

leatherworking_temp = leatherworking_temp.sort_values(by=['id', 'reagents_str', 'character'], 
                                                      ascending=[True, True, True])
leatherworking_temp = leatherworking_temp.drop_duplicates(subset=['id', 'skill1', 'reagents_str'], keep='first')

leatherworking_temp = leatherworking_temp.sort_values(by=['id', 'reagents_str', 'skill1', 'character'], 
                                                      ascending=[True, True, False, True])
leatherworking_temp = leatherworking_temp.drop_duplicates(subset=['id', 'reagents_str'], keep='first')

leatherworking_temp = leatherworking_temp.drop('reagents_str', axis=1)

leatherworking = Profession('Leatherworking')
leatherworking.set_table(leatherworking_temp)

### Tailoring

In [18]:
# Last Updates:
# Trees: 140/140 && 110/110 && 130/130 && 250/250
# Gear: 73 Insp + 49 Multi + 10 Skill && 44 Res + 44 Speed + 6 Skill && (110 Multi + 45 Res || 110 Insp + 40 Insp) + 10 Skill

tailoring = Profession('Tailoring')

### Woven Cloth ###
tailoring.add('Azureweave Bolt', 193940, {190329:1, 190324:1, 193931:3}, 'Linidel', 'Crafting Reagent', difficulty=425, insp_percent=40.3, multicraft=13.5, quantity=1, skill=301, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Chronocloth Bolt', 193937, {190327:1, 190324:1, 193931:3}, 'Linidel', 'Crafting Reagent', difficulty=425, insp_percent=40.3, multicraft=13.5, quantity=1, skill=301, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Vibrant Wildercloth Bolt', 193931, {194124:1, 193928:3}, 'Linidel', 'Crafting Reagent', difficulty=300, insp_percent=37.3, multicraft=11.7, quantity=1, skill=216, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Infurious Wildercloth Bolt', 193934, {190450:1, 193922:3}, 'Linidel', 'Crafting Reagent', difficulty=200, insp_percent=22.3, multicraft=21.7, quantity=1, skill=216, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Wildercloth Bolt', 193928, {193922:3}, 'Linidel', 'Crafting Reagent', difficulty=60, insp_percent=22.3, multicraft=21.7, quantity=1, skill=216, rarity='uncommon', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Optional Reagents ###
tailoring.add('Blue Silken Lining', 193946, {201405:1, 192097:5, 193940:1}, 'Linidel', 'Embellishment', difficulty=325, insp_percent=37.3, multicraft=4.5, quantity=1, skill=236, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Bronzed Grip Wrappings', 193943, {192097:15, 193937:1}, 'Linidel', 'Embellishment', difficulty=325, insp_percent=37.3, multicraft=4.5, quantity=1, skill=236, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Medical Wrap Kit', 205411, {193922:50, 204460:200}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=14.5, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Reserve Parachute', 205012, {193922:100, 204460:200, 190327:10, 192097:5}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=14.5, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Verdant Tether', 210673, {201405:1, 208212:3, 192097:25, 193931:5}, 'Linidel', 'Embellishment', difficulty=325, insp_percent=37.3, multicraft=4.5, quantity=1, skill=236, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Finishing Reagents ###
tailoring.add('Abrasive Polishing Cloth', 193952, {192872:3, 193928:1}, 'Linidel', 'Crafting Reagent', difficulty=200, insp_percent=22.3, multicraft=14.5, quantity=2, skill=236, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Illustrious Insight', 191529, {190456:50}, 'Linidel', 'Crafting Reagent', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Vibrant Polishing Cloth', 193955, {192097:1, 193931:1}, 'Linidel', 'Crafting Reagent', difficulty=300, insp_percent=37.3, multicraft=4.5, quantity=2, skill=236, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Chromatic Embroidery Thread', 193961, {194123:5, 192097:1}, 'Linidel', 'Crafting Reagent', difficulty=300, insp_percent=35.3, multicraft=4.5, quantity=2, skill=236, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Shimmering Embroidery Thread', 193964, {192097:1, 193370:1}, 'Linidel', 'Crafting Reagent', difficulty=300, insp_percent=35.3, multicraft=4.5, quantity=2, skill=236, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Blazing Embroidery Thread', 193958, {190321:1, 192097:1}, 'Linidel', 'Crafting Reagent', difficulty=200, insp_percent=20.3, multicraft=14.5, quantity=2, skill=236, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Garments ###
tailoring.add('Undulating Sporecloak', 205025, {206959:1, 204460:300, 192097:8, 193931:12}, 'Linidel', 'Gear (Combat)', difficulty=310, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True)
tailoring.add('Vibrant Wildercloth Girdle', 193516, {206959:1, 192097:6, 193931:10}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Handwraps', 193504, {206959:1, 192097:8, 193931:10}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Headcover', 193523, {206959:1, 192097:10, 193931:12}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Shawl', 193511, {206959:1, 192097:6, 193931:8}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Shoulderspikes', 193508, {206959:1, 192097:8, 193931:10}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Slacks', 193518, {206959:1, 192097:10, 193931:12}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Slippers', 193519, {206959:1, 192097:8, 193931:10}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Vestments', 193509, {206959:1, 192097:10, 193931:12}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Vibrant Wildercloth Wristwraps', 193510, {206959:1, 192097:6, 193931:8}, 'Linidel', 'Gear (Combat)', difficulty=280, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=True, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Surveyor's Seasoned Hood", 201936, {193922:10, 192097:7}, 'Linidel', 'Gear (Combat)', difficulty=60, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Obsidian Combatant's Wildercloth Bands", 204115, {192097:6, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Cloak", 204120, {192097:8, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Gloves", 204117, {192097:8, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Hood", 204116, {192097:10, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Leggings", 204112, {192097:10, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Sash", 204119, {192097:6, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Shoulderpads", 204113, {192097:8, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Treads", 204114, {192097:8, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Obsidian Combatant's Wildercloth Tunic", 204118, {192097:10, 193934:4}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Surveyor's Seasoned Pants", 201939, {193922:10, 192097:7}, 'Linidel', 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Verdant Combatant's Wildercloth Bands", 208258, {192097:10, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Cloak", 208263, {192097:12, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Gloves", 208260, {192097:12, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Hood", 208259, {192097:14, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Leggings", 208255, {192097:14, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Sash", 208262, {192097:10, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Shoulderpads", 208256, {192097:12, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Treads", 208257, {192097:12, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Verdant Combatant's Wildercloth Tunic", 208261, {192097:14, 193934:6}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("surveyor's Seasoned Shoulders", 201937, {193922:8, 192097:6}, 'Linidel', 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Crimson Combatant's Wildercloth Bands", 193505, {192097:3, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Cloak", 193522, {192097:4, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Gloves", 193507, {192097:4, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Hood", 193506, {192097:5, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Leggings", 193501, {192097:5, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Sash", 193517, {192097:3, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Shoulderpads", 193502, {192097:4, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Treads", 193503, {192097:4, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Crimson Combatant's Wildercloth Tunic", 193514, {192097:5, 193934:2}, 'Linidel', 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Surveyor's Seasoned Gloves", 201938, {193922:8, 192097:6}, 'Linidel', 'Gear (Combat)', difficulty=60, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Surveyor's Seasoned Cord", 201940, {193922:8, 192097:6}, 'Linidel', 'Gear (Combat)', difficulty=60, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Surveyor's Tailored Cloak", 193500, {193922:8, 192097:6}, 'Linidel', 'Gear (Combat)', difficulty=60, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Surveyor's Cloth Robe", 193515, {193922:8, 192097:5}, 'Linidel', 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Surveyor's Cloth Treads", 193498, {193922:6, 192097:5}, 'Linidel', 'Gear (Combat)', difficulty=40,  insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add("Surveyor's Cloth Bands", 193497, {193922:4, 192097:4}, 'Linidel', 'Gear (Combat)', difficulty=40, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=True) 

### Azureweave Garments ###
tailoring.add('Amice of the Blue', 193526, {206959:1, 193919:1, 194013:1, 193940:3}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Azureweave Mantle', 193520, {206959:1, 194013:1, 193940:3}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Azureweave Robe', 193536, {206959:1, 194013:1, 193940:4}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Azureweave Slippers', 193525, {206959:1, 194013:1, 193940:3}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=41.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Blue Dragon Soles', 193532, {206959:1, 193919:1, 194013:1, 193940:3}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=41.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Infurious Binding of Gesticulation', 193513, {206959:1, 201404:2, 194010:1, 193940:1, 193934:16}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 

### Chronocloth Garments ###
tailoring.add('Allied Wristguards of Time Dilation', 193530, {206959:1, 193360:1, 194016:1, 193937:2}, 'Linidel',  'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Chronocloth Gloves', 193527, {206959:1, 194016:1, 193937:3}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=41.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Chronocloth Leggings', 193537, {206959:1, 194016:1, 193937:4}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Chronocloth Sash', 193512, {206959:1, 194016:1, 193937:3}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Hood of Surging Time', 193521, {206959:1, 193921:1, 194016:1, 193937:4}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=41.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 
tailoring.add('Infurious Legwraps of Possibility', 193524, {206959:1, 194010:1, 193937:1, 193934:20}, 'Linidel', 'Gear (Combat)', difficulty=400, insp_percent=40.3, multicraft=np.nan, quantity=1, skill=336, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=True) 

### Profession Garments ###
tailoring.add("Dragoncloth Tailoring Vestments", 193540, {190456:325, 194010:1, 193937:3, 193940:3}, 'Linidel', 'Gear (Profession)', difficulty=475, insp_percent=42.3, multicraft=np.nan, quantity=1, skill=366, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Master's Wildercloth Alchemist's Robe", 193544, {190456:225, 192097:5, 193931:6, 191498:10}, 'Linidel', 'Gear (Profession)', difficulty=400, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Master's Wildercloth Chef's Hat", 193545, {190456:225, 192097:6, 193931:6}, 'Linidel', 'Gear (Profession)', difficulty=400, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Master's Wildercloth Enchanter's Hat", 193533, {190456:225, 200113:2, 192097:5, 193931:5}, 'Linidel', 'Gear (Profession)', difficulty=400, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Master's Wildercloth Fishing Cap", 193543, {198397:2, 192097:5, 193931:3}, 'Linidel', 'Gear (Profession)', difficulty=400, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Master's Wildercloth Gardening Hat", 193542, {190456:225, 192097:5, 193931:6, 191498:4}, 'Linidel', 'Gear (Profession)', difficulty=400, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Wildercloth Enchanter's Hat", 193539, {193922:3, 194123:3, 192097:4}, 'Linidel', 'Gear (Profession)', difficulty=80, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Wildercloth Alchemist's Robe", 193528, {193922:6, 192097:4}, 'Linidel', 'Gear (Profession)', difficulty=80, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Wildercloth Fishing Cap", 193529, {193922:5, 192097:4}, 'Linidel', 'Gear (Profession)', difficulty=80, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Wildercloth Chef's Hat", 193534, {193922:5, 192097:4}, 'Linidel', 'Gear (Profession)', difficulty=80, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Wildercloth Gardening Hat", 193538, {193922:5, 192097:4, 191462:5}, 'Linidel', 'Gear (Profession)', difficulty=80, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Wildercloth Tailor's Coat", 193541, {193922:6, 192097:4}, 'Linidel', 'Gear (Profession)', difficulty=80, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Spellthread ###
tailoring.add('Frozen Spellthread', 194013, {190329:1, 190324:2, 193940:1}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=38.3, multicraft=4.5, quantity=1, skill=296, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Temporal Spellthread', 194016, {190327:1, 190324:2, 193937:1}, 'Linidel', 'Item Enhancement', difficulty=425, insp_percent=38.3, multicraft=4.5, quantity=1, skill=296, rarity='epic', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Vibrant Spellthread', 194010, {194124:1, 193931:2}, 'Linidel', 'Item Enhancement', difficulty=150, insp_percent=20.3, multicraft=14.5, quantity=1, skill=236, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Embroidered Bags ###
tailoring.add('Azureweave Expedition Pack', 194018, {192097:5, 193940:3}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Chronocloth Reagent Bag', 194020, {192097:6, 193937:6}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Wildercloth Bag', 194017, {192097:4, 193928:12}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Simply Stitched Reagent Bag', 194019, {192097:4, 193928:8}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Assorted Embroidery ###
tailoring.add("Explorer's Banner of Geology", 194047, {192097:2, 193928:2, 192887:2}, 'Linidel', 'Other', difficulty=325, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=236, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Explorer's Banner of Herbology", 194044, {192097:2, 193931:7, 191498:12}, 'Linidel', 'Other', difficulty=325, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=236, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Forlorn Funeral Pall', 194052, {190331:5, 192097:4, 193931:12}, 'Linidel', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Duck-Stuffed Duck Lovie', 194056, {193053:75, 192097:8, 193931:6}, 'Linidel', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Dragonscale Expedition's Expedition Tent", 194060, {201404:4, 192097:20, 193928:40}, 'Linidel', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Cold Cushion', 194058, {193053:25, 201401:5, 192097:5, 193940:1}, 'Linidel', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Market Tent', 194059, {192097:20, 193928:50}, 'Linidel', 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Wildercloth Bandage', 194050, {193922:2}, 'Linidel', 'Consumable', difficulty=40, insp_percent=22.3, multicraft=14.5, quantity=1, skill=236, rarity='common', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Artisan Curious ###
tailoring.add('Traditional Morqut Kite', 203415, {203406:1}, 'Linidel', 'Other', difficulty=np.nan, insp_percent=np.nan, multicraft=14.5, quantity=1, skill=np.nan, rarity='epic', has_reagent_qualities=False, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)

### Unlearned ###
tailoring.add('Paw-Made Winterpelt Reagent Bag', 202287, {193928:144, 192097:36, 193218:4}, None, 'Other', difficulty=np.nan, insp_percent= np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add('Cushion of Time Travel', 194057, {193053:25, 201401:5, 192097:5, 193937:1}, None, 'Toy', difficulty=np.nan, insp_percent=np.nan, multicraft=np.nan, quantity=1, skill=np.nan, rarity='rare', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=False, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Bands", 211557, {192097:10, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Cloak", 211562, {192097:12, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Gloves", 211559, {192097:12, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Hood", 211558, {192097:14, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Leggings", 211554, {192097:14, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Sash", 211561, {192097:10, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Shoulderpads", 211555, {192097:12, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Treads", 211556, {192097:12, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=38.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)
tailoring.add("Draconic Combatant's Wildercloth Tunic", 211560, {192097:14, 193934:6}, None, 'Gear (Combat)', difficulty=120, insp_percent=37.3, multicraft=np.nan, quantity=1, skill=276, rarity='uncommon', has_reagent_qualities=True, has_embellishment_slot=False, has_missive_slot=True, has_tinker_slot=False, has_crest_slot=False)


### Sub-Final Frames

In [19]:
all_data = pd.concat((alchemy.get_table(), blacksmithing.get_table(), cooking.get_table(), enchanting.get_table(), 
                      engineering.get_table(), inscription.get_table(), jewelcrafting.get_table(), 
                      leatherworking.get_table(), tailoring.get_table()), ignore_index=True)

In [20]:
columns = ['id', 'item', 'icon']
items = all_data.loc[:, ['id', 'item', 'icon']]
items = items.drop_duplicates()
        
for index, row in tqdm(all_data.iterrows(), total=len(all_data)):
    for reagent in row['reagents'].keys():
        if reagent not in items.loc[:, 'id'].to_numpy():
            url = f'https://www.wowhead.com/item={reagent}?xml'
            html = requests.get(url).text
            soup = BeautifulSoup(html, features='xml')
            name = soup.find('name').text
            df = pd.DataFrame(columns=columns, data=[[reagent, name, None]])
            items = pd.concat((items, df))

100%|██████████| 908/908 [00:43<00:00, 20.73it/s] 


In [21]:
items

,id,item,icon
0,191495,Primal Convergent,NaN
1,191498,Omnium Draconis,NaN
2,191374,Residual Neural Channeling Agent,NaN
3,204370,Stinky Bright Potion,NaN
4,191362,Bottled Putrescence,NaN
...,...,...,...
0,201404,Tallstrider Sinew,NaN
0,203405,Pristine Pelt,NaN
0,192097,Spool of Wilderthread,NaN
0,198397,Rainbow Pearl,NaN


## Finalization

### Add regeants to items frame

### Icon Link Scraping

In [22]:
icon_links = list()

for x,y in tqdm(items.iterrows(), total=len(items)):
    item_id, _, _ = y
    #item_id, item_name, item_icon, item_tag, item_rarity = y
    url = f'https://www.wowhead.com/item={item_id}'
    soup = BeautifulSoup(requests.get(url).text)
    
    #wowhead no longer has this href so need to us another method
    #link = soup.find(name='link', attrs={'rel':'image_src'})['href'] 
    
    #if link.find('http') < 0:
    #    icon_links.append(None)
    #    continue
    #    
    #first = link.find('http', 1)
    #second = link.find('http', 2)
    #if second >= first:
    #    icon_links.append(link[second:])
    #else:
    #    assert('what')
    
    #new method using regex
    
    #string1 finds strings preceded by:   "{item_id}:{" 
    #and are also followed by:    ,"screenshot"
    #the strings cannot include the symbol:   }
    string1 = re.search(r'(?<="'+f'{item_id}'+r'":{)[^}]+(?=,"screenshot")', str(soup)).group()
    
    #string2 searches string1 for strings preceded by:      "icon":"
    #and are also followed by:      ")
    #that only contain a-z, A-Z, 0-9, _, and -
    string2 = re.search(r'(?<="icon":")[\w-]+(?=")', string1).group()
    site = 'https://wow.zamimg.com/images/wow/icons/large/'+string2+'.jpg'
    
    status_code = requests.get(site).status_code
    if status_code==200:
        icon_links.append(site)
    else:
        icon_links.append(None)
        print(item_id)
    
items.loc[:,'icon'] = icon_links

100%|██████████| 1052/1052 [37:32<00:00,  2.14s/it] 


In [23]:
items.iloc[175]

id                                                 211570
item             Draconic Combatant's Draconium Armguards
icon    https://wow.zamimg.com/images/wow/icons/large/...
Name: 176, dtype: object

In [24]:
string2 = re.search(r'(?<="icon":")[\w-]+(?=")', string1).group()

In [25]:
icon_links

['https://wow.zamimg.com/images/wow/icons/large/inv_10_elementalshardfoozles_primordial.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_misc_herb_chamlotus.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_10_alchemy_bottle_shape1_yellow.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_trade_alchemy_dpotion_c1a.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_10_alchemy_bottle_shape1_black.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_10_alchemy_bottle_shape1_green.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_10_alchemy_bottle_shape1_violet.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_10_alchemy_bottle_shape1_white.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_10_alchemy_bottle_shape1_blue.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_enchant_alchemistcauldron.jpg',
 'https://wow.zamimg.com/images/wow/icons/large/inv_enchanting_70_pet_cauldron.jpg',
 'https://wow.zamimg.com/images/wow/icon

In [26]:
#update icons in full frame
all_data = pd.merge(left=all_data.drop('icon', axis=1), right=items[['id', 'icon']], how='left', on='id')
all_data

,profession,character,id,item,reagents,tag,rarity,difficulty,skill1,hsv,inspiration_skill,inspiration_percent,base_quantity,multicraft_percent,has_reagent_qualities,has_embellishment_slot,has_missive_slot,has_tinker_slot,has_crest_slot,icon
0,Alchemy,Trillithia,191495,Primal Convergent,"{190316: 1, 190321: 1, 190327: 1, 190329: 1, 1...",Crafting Reagent,uncommon,275.0,222.0,13.75,137.5,34.8,2,8.2,False,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
1,Alchemy,Trillithia,191498,Omnium Draconis,"{191472: 1, 191466: 1, 191462: 1, 191469: 1}",Crafting Reagent,uncommon,325.0,222.0,16.25,162.5,34.8,1,8.2,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
2,Alchemy,Trillithia,191374,Residual Neural Channeling Agent,"{190327: 1, 190316: 1, 191476: 1, 191466: 1}",Consumable,common,400.0,272.0,20.00,200.0,35.8,5,15.9,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
3,Alchemy,Trillithia,204370,Stinky Bright Potion,"{190326: 1, 190330: 1, 204460: 50, 191476: 5, ...",Consumable,common,NaN,NaN,NaN,NaN,NaN,5,25.9,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
4,Alchemy,Trillithia,191362,Bottled Putrescence,"{190327: 1, 190331: 1, 191476: 5, 191462: 30}",Consumable,common,450.0,312.0,22.50,225.0,36.8,5,15.9,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,Tailoring,<NA>,211554,Draconic Combatant's Wildercloth Leggings,"{192097: 14, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,60.0,37.3,1,NaN,True,False,True,False,False,https://wow.zamimg.com/images/wow/icons/large/...
916,Tailoring,<NA>,211561,Draconic Combatant's Wildercloth Sash,"{192097: 10, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,60.0,37.3,1,NaN,True,False,True,False,False,https://wow.zamimg.com/images/wow/icons/large/...
917,Tailoring,<NA>,211555,Draconic Combatant's Wildercloth Shoulderpads,"{192097: 12, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,60.0,37.3,1,NaN,True,False,True,False,False,https://wow.zamimg.com/images/wow/icons/large/...
918,Tailoring,<NA>,211556,Draconic Combatant's Wildercloth Treads,"{192097: 12, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,60.0,38.3,1,NaN,True,False,True,False,False,https://wow.zamimg.com/images/wow/icons/large/...


In [27]:
df = all_data.loc[all_data['icon'].isna(), ['id', 'icon']]
display(df)

#ensure all items have icons
assert(len(df)==0)

,id,icon


### Update Item IDs to be Rank 3

In [28]:
old_ids = items['id'].astype(int).to_numpy()
new_ids = [check_id(old_id) for old_id in tqdm(old_ids)]
items['update_id'] = new_ids

100%|██████████| 1052/1052 [07:05<00:00,  2.47it/s]


In [29]:
print(*items.loc[items['update_id']==1, 'id'].to_numpy(), sep='\n')

In [30]:
to_replace = dict(zip(items.loc[(items['update_id']!=-1)&(items['update_id']!=items['id']), 'id'].to_numpy(), 
                      items.loc[(items['update_id']!=-1)&(items['update_id']!=items['id']), 'update_id'].to_numpy()))

new_keys = list(to_replace.keys())
new_vals = list(to_replace.values())

replacements = dict(zip(new_keys, new_vals))
display(replacements)

#ensure all items are rank 3 where possible
assert(len(replacements.keys())==0)

items = items.drop('update_id', axis=1)

{}

### Add Crafting Odds

In [31]:
all_data.head()

,profession,character,id,item,reagents,tag,rarity,difficulty,skill1,hsv,inspiration_skill,inspiration_percent,base_quantity,multicraft_percent,has_reagent_qualities,has_embellishment_slot,has_missive_slot,has_tinker_slot,has_crest_slot,icon
0,Alchemy,Trillithia,191495,Primal Convergent,"{190316: 1, 190321: 1, 190327: 1, 190329: 1, 1...",Crafting Reagent,uncommon,275.0,222.0,13.75,137.5,34.8,2,8.2,False,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
1,Alchemy,Trillithia,191498,Omnium Draconis,"{191472: 1, 191466: 1, 191462: 1, 191469: 1}",Crafting Reagent,uncommon,325.0,222.0,16.25,162.5,34.8,1,8.2,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
2,Alchemy,Trillithia,191374,Residual Neural Channeling Agent,"{190327: 1, 190316: 1, 191476: 1, 191466: 1}",Consumable,common,400.0,272.0,20.00,200.0,35.8,5,15.9,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
3,Alchemy,Trillithia,204370,Stinky Bright Potion,"{190326: 1, 190330: 1, 204460: 50, 191476: 5, ...",Consumable,common,NaN,NaN,NaN,NaN,NaN,5,25.9,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...
4,Alchemy,Trillithia,191362,Bottled Putrescence,"{190327: 1, 190331: 1, 191476: 5, 191462: 30}",Consumable,common,450.0,312.0,22.50,225.0,36.8,5,15.9,True,False,False,False,False,https://wow.zamimg.com/images/wow/icons/large/...


In [32]:
### initialization ###

#skill with rank 2 mats
all_data['skill2'] = all_data['skill1']+all_data['difficulty']*0.125

#skill with rank 3 mats
all_data['skill3'] = all_data['skill1']+all_data['difficulty']*0.25

#tinker+embellishment not currently possible
all_data['difficulty_tinker'] = all_data['difficulty']+10
all_data['difficulty_missive'] = all_data['difficulty']+15
all_data['difficulty_embellishment'] = all_data['difficulty']+25
all_data['difficulty_tinker_missive'] = all_data['difficulty']+25
#all_data['difficulty_tinker_embellishment'] = all_data['difficulty']+35
all_data['difficulty_missive_embellishment'] = all_data['difficulty']+40
#all_data['difficulty_tinker_missive_embellishment'] = all_data['difficulty']+50
all_data['difficulty_whelpling'] = all_data['difficulty']+160
all_data['difficulty_whelpling_tinker'] = all_data['difficulty']+170
all_data['difficulty_whelpling_missive'] = all_data['difficulty']+175
all_data['difficulty_whelpling_embellishment'] = all_data['difficulty']+185
all_data['difficulty_whelpling_tinker_missive'] = all_data['difficulty']+185
#all_data['difficulty_whelpling_tinker_embellishment'] = all_data['difficulty']+195
all_data['difficulty_whelpling_missive_embellishment'] = all_data['difficulty']+200
#all_data['difficulty_whelpling_tinker_missive_embellishment'] = all_data['difficulty']+210
all_data['difficulty_wyrm'] = all_data['difficulty']+30
all_data['difficulty_wyrm_tinker'] = all_data['difficulty']+40
all_data['difficulty_wyrm_missive'] = all_data['difficulty']+45
all_data['difficulty_wyrm_embellishment'] = all_data['difficulty']+55
all_data['difficulty_wyrm_tinker_missive'] = all_data['difficulty']+55
#all_data['difficulty_wyrm_tinker_embellishment'] = all_data['difficulty']+65
all_data['difficulty_wyrm_missive_embellishment'] = all_data['difficulty']+70
#all_data['difficulty_wyrm_tinker_missive_embellishment'] = all_data['difficulty']+80
all_data['difficulty_aspect'] = all_data['difficulty']+50
all_data['difficulty_aspect_tinker'] = all_data['difficulty']+60
all_data['difficulty_aspect_missive'] = all_data['difficulty']+65
all_data['difficulty_aspect_embellishment'] = all_data['difficulty']+75
all_data['difficulty_aspect_tinker_missive'] = all_data['difficulty']+75
#all_data['difficulty_aspect_tinker_embellishment'] = all_data['difficulty']+85
all_data['difficulty_aspect_missive_embellishment'] = all_data['difficulty']+90
#all_data['difficulty_aspect_tinker_missive_embellishment'] = all_data['difficulty']+100

modifiers = ['', '_tinker', '_missive', '_embellishment', '_tinker_missive', '_missive_embellishment', '_whelpling',
             '_whelpling_tinker', '_whelpling_tinker_missive', '_whelpling_missive', '_whelpling_embellishment',
             '_whelpling_missive_embellishment', '_wyrm', '_wyrm_tinker', '_wyrm_tinker_missive', '_wyrm_missive',
             '_wyrm_embellishment', '_wyrm_missive_embellishment', '_aspect', '_aspect_tinker', '_aspect_missive',
             '_aspect_tinker_missive', '_aspect_embellishment', '_aspect_missive_embellishment']

### no additions ###
for modifier in modifiers:
    difficulty = 'difficulty'+modifier
    for i in range(3):
        odds_col = 'odds'+modifier+str(i+1)
        odds_hsv_col = 'odds'+modifier+'_hsv'+str(i+1)
        skill_col = 'skill'+str(i+1)

        all_data[odds_col] = all_data['inspiration_percent']
        all_data.loc[all_data[difficulty] < all_data[skill_col], odds_col] = 100.0

        all_data['skill_diff'] = all_data[difficulty]+1-all_data[skill_col]
        all_data['hsv_leftover'] = all_data['hsv']-all_data['skill_diff']
        all_data.loc[(all_data['skill_diff']<0)|(all_data['hsv_leftover']<0), 'hsv_leftover'] = 0
        all_data['hsv_percent'] = 100*all_data['hsv_leftover']/(100*all_data['hsv']+1) #calculate to a precision of 1/100, which allows for the possibility of rolling a 0
        all_data.loc[all_data['hsv_leftover']==0, 'hsv_percent'] = 0
        all_data[odds_hsv_col] = all_data[odds_col]+(100-all_data['inspiration_percent'])*all_data['hsv_percent']
        all_data.loc[all_data[odds_hsv_col]>100, odds_hsv_col] = 100.0
        all_data = all_data.drop(['skill_diff', 'hsv_leftover', 'hsv_percent'], axis=1)

        for col in [odds_col, odds_hsv_col]:
            all_data[col] = all_data[col].apply(np.round, args=(1,))
    
### final rounding ###
    
all_data['skill2'] = all_data['skill2'].apply(np.round, args=(1,))
all_data['skill3'] = all_data['skill3'].apply(np.round, args=(1,))

all_data

,profession,character,id,item,reagents,tag,rarity,difficulty,skill1,hsv,...,odds_aspect_embellishment2,odds_aspect_embellishment_hsv2,odds_aspect_embellishment3,odds_aspect_embellishment_hsv3,odds_aspect_missive_embellishment1,odds_aspect_missive_embellishment_hsv1,odds_aspect_missive_embellishment2,odds_aspect_missive_embellishment_hsv2,odds_aspect_missive_embellishment3,odds_aspect_missive_embellishment_hsv3
0,Alchemy,Trillithia,191495,Primal Convergent,"{190316: 1, 190321: 1, 190327: 1, 190329: 1, 1...",Crafting Reagent,uncommon,275.0,222.0,13.75,...,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8
1,Alchemy,Trillithia,191498,Omnium Draconis,"{191472: 1, 191466: 1, 191462: 1, 191469: 1}",Crafting Reagent,uncommon,325.0,222.0,16.25,...,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8
2,Alchemy,Trillithia,191374,Residual Neural Channeling Agent,"{190327: 1, 190316: 1, 191476: 1, 191466: 1}",Consumable,common,400.0,272.0,20.00,...,35.8,35.8,35.8,35.8,35.8,35.8,35.8,35.8,35.8,35.8
3,Alchemy,Trillithia,204370,Stinky Bright Potion,"{190326: 1, 190330: 1, 204460: 50, 191476: 5, ...",Consumable,common,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alchemy,Trillithia,191362,Bottled Putrescence,"{190327: 1, 190331: 1, 191476: 5, 191462: 30}",Consumable,common,450.0,312.0,22.50,...,36.8,36.8,36.8,36.8,36.8,36.8,36.8,36.8,36.8,36.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,Tailoring,<NA>,211554,Draconic Combatant's Wildercloth Leggings,"{192097: 14, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
916,Tailoring,<NA>,211561,Draconic Combatant's Wildercloth Sash,"{192097: 10, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
917,Tailoring,<NA>,211555,Draconic Combatant's Wildercloth Shoulderpads,"{192097: 12, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
918,Tailoring,<NA>,211556,Draconic Combatant's Wildercloth Treads,"{192097: 12, 193934: 6}",Gear (Combat),uncommon,120.0,276.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


### Remove pd.NA and Sorting

In [33]:
all_data = all_data.reset_index(drop=True)

all_data['character'] = all_data['character'].replace({pd.NA:None})
all_data = all_data.sort_values(by=['profession', 'item'], ascending=[True, True])

display(all_data['character'].unique())
assert(len(all_data['character'].unique()==8) or len(all_data['character'].unique()==9))

array(['Trillithia', 'Zarastannil', None, 'Linidel', 'Mellasona',
       'Nystelil', 'Braevele'], dtype=object)

### File Saving

In [34]:
items.to_pickle('../../static/data/items_DF.pkl')

In [35]:
all_data.to_pickle('../../static/data/data_DF.pkl')

In [4]:
items = pd.read_pickle('../../static/data/items_DF.pkl')
all_data = pd.read_pickle('../../static/data/data_DF.pkl')

In [37]:
all_data.loc[all_data.loc[:,'item']=='Torc of Passed Time',:]

,profession,character,id,item,reagents,tag,rarity,difficulty,skill1,hsv,...,odds_aspect_embellishment2,odds_aspect_embellishment_hsv2,odds_aspect_embellishment3,odds_aspect_embellishment_hsv3,odds_aspect_missive_embellishment1,odds_aspect_missive_embellishment_hsv1,odds_aspect_missive_embellishment2,odds_aspect_missive_embellishment_hsv2,odds_aspect_missive_embellishment3,odds_aspect_missive_embellishment_hsv3
605,Jewelcrafting,Nystelil,201759,Torc of Passed Time,"{206959: 1, 192836: 2, 192887: 1, 192858: 1}",Gear (Combat),epic,280.0,272.0,14.0,...,36.3,36.3,36.3,36.3,36.3,36.3,36.3,36.3,36.3,36.3


In [38]:
jewelcrafting.get_table()

,profession,character,id,item,icon,reagents,tag,rarity,difficulty,skill1,hsv,inspiration_skill,inspiration_percent,base_quantity,multicraft_percent,has_reagent_qualities,has_embellishment_slot,has_missive_slot,has_tinker_slot,has_crest_slot
0,Jewelcrafting,Nystelil,208748,Dreamtender's Charm,<NA>,"{208212: 1, 210456: 1, 192836: 1, 192861: 2, 1...",Embellishment,rare,340.0,227.0,17.00,170.0,36.3,1,5.0,True,False,False,False,False
1,Jewelcrafting,Nystelil,193379,Elemental Harmony,<NA>,"{192852: 1, 192858: 1, 192861: 1, 192862: 1, 1...",Crafting Reagent,rare,275.0,227.0,13.75,137.5,36.3,2,15.0,True,False,False,False,False
2,Jewelcrafting,Nystelil,191529,Illustrious Insight,<NA>,{190456: 50},Crafting Reagent,rare,NaN,NaN,NaN,NaN,NaN,1,NaN,False,False,False,False,False
3,Jewelcrafting,Nystelil,192896,Blotting Sand,<NA>,"{190320: 1, 193922: 1, 193370: 1}",Crafting Reagent,uncommon,225.0,227.0,11.25,112.5,21.3,2,15.0,False,False,False,False,False
4,Jewelcrafting,Nystelil,192899,Pounce,<NA>,"{190315: 2, 193922: 1, 193370: 2}",Crafting Reagent,uncommon,275.0,227.0,13.75,137.5,36.3,2,5.0,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Jewelcrafting,<NA>,211536,Draconic Combatant's Jeweled Signet,<NA>,"{190451: 8, 192836: 1, 192842: 5}",Gear (Combat),uncommon,120.0,272.0,6.00,60.0,36.3,1,NaN,True,False,True,False,False
89,Jewelcrafting,<NA>,193885,Jeweled Amber Whelpling,<NA>,"{200863: 1, 192885: 50, 192868: 2, 192887: 2, ...",Pet,rare,NaN,NaN,NaN,NaN,NaN,1,NaN,True,False,False,False,False
90,Jewelcrafting,<NA>,193888,Jeweled Emerald Whelpling,<NA>,"{200865: 1, 192885: 50, 192861: 2, 192887: 2, ...",Pet,rare,NaN,NaN,NaN,NaN,NaN,1,NaN,True,False,False,False,False
91,Jewelcrafting,<NA>,193887,Jeweled Ruby Whelpling,<NA>,"{200864: 1, 192885: 50, 192852: 2, 192887: 2, ...",Pet,rare,NaN,NaN,NaN,NaN,NaN,1,NaN,True,False,False,False,False


In [39]:
items[items['icon'].isna()]

,id,item,icon


In [40]:
items.loc[items['item']=='Shadowed Belt Clasp','icon'].iloc[0]

'https://wow.zamimg.com/images/wow/icons/large/inv_10_skinning_craftedoptionalreagent_armorbuckles_color2.jpg'

In [9]:
np.unique(all_data[all_data['profession']=='Alchemy']['tag'])

array(['Consumable', 'Crafting Reagent', 'Embellishment', 'Gear (Combat)',
       'Other'], dtype=object)

In [11]:
list(all_data.columns)

['profession',
 'character',
 'id',
 'item',
 'reagents',
 'tag',
 'rarity',
 'difficulty',
 'skill1',
 'hsv',
 'inspiration_skill',
 'inspiration_percent',
 'base_quantity',
 'multicraft_percent',
 'has_reagent_qualities',
 'has_embellishment_slot',
 'has_missive_slot',
 'has_tinker_slot',
 'has_crest_slot',
 'icon',
 'skill2',
 'skill3',
 'difficulty_tinker',
 'difficulty_missive',
 'difficulty_embellishment',
 'difficulty_tinker_missive',
 'difficulty_missive_embellishment',
 'difficulty_whelpling',
 'difficulty_whelpling_tinker',
 'difficulty_whelpling_missive',
 'difficulty_whelpling_embellishment',
 'difficulty_whelpling_tinker_missive',
 'difficulty_whelpling_missive_embellishment',
 'difficulty_wyrm',
 'difficulty_wyrm_tinker',
 'difficulty_wyrm_missive',
 'difficulty_wyrm_embellishment',
 'difficulty_wyrm_tinker_missive',
 'difficulty_wyrm_missive_embellishment',
 'difficulty_aspect',
 'difficulty_aspect_tinker',
 'difficulty_aspect_missive',
 'difficulty_aspect_embellishment'

In [13]:
all_data[(all_data['character']!='Trillithia')&(all_data['profession']=='Engineering')]

,profession,character,id,item,reagents,tag,rarity,difficulty,skill1,hsv,...,odds_aspect_embellishment2,odds_aspect_embellishment_hsv2,odds_aspect_embellishment3,odds_aspect_embellishment_hsv3,odds_aspect_missive_embellishment1,odds_aspect_missive_embellishment_hsv1,odds_aspect_missive_embellishment2,odds_aspect_missive_embellishment_hsv2,odds_aspect_missive_embellishment3,odds_aspect_missive_embellishment_hsv3
410,Engineering,None,198255,Calibrated Safety Switch,"{198197: 1, 198188: 2, 198194: 1, 198200: 1}",Crafting Reagent,uncommon,425.0,297.0,21.25,...,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8
411,Engineering,None,198261,Critical Failure Prevention Unit,"{198197: 1, 198280: 2, 198188: 3, 198200: 1}",Crafting Reagent,uncommon,425.0,297.0,21.25,...,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8
414,Engineering,None,211612,Draconic Combatant's Cloth Goggles,"{193934: 6, 192878: 2, 198185: 4}",Gear (Combat),uncommon,120.0,302.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
415,Engineering,None,211613,Draconic Combatant's Leather Goggles,"{193238: 4, 192878: 2, 198185: 4}",Gear (Combat),uncommon,120.0,302.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
416,Engineering,None,211614,Draconic Combatant's Mail Goggles,"{193250: 4, 192878: 2, 198185: 4}",Gear (Combat),uncommon,120.0,302.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
417,Engineering,None,211615,Draconic Combatant's Plate Goggles,"{190538: 6, 192878: 2, 198185: 4}",Gear (Combat),uncommon,120.0,302.0,6.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
419,Engineering,None,198209,EZ-Thro Creature Combustion Canister,"{198191: 6, 198188: 1, 198185: 4, 198203: 1}",Consumable,uncommon,250.0,327.0,12.50,...,100.0,100.0,100.0,100.0,16.8,16.8,100.0,100.0,100.0,100.0
420,Engineering,None,198215,EZ-Thro Gravitational Displacer,"{190330: 1, 198191: 4, 198188: 1, 198185: 2, 1...",Consumable,uncommon,350.0,327.0,17.50,...,31.8,31.8,31.8,55.2,31.8,31.8,31.8,31.8,31.8,31.8
421,Engineering,None,199195,EZ-Thro Grease Grenade,"{190330: 1, 198203: 1, 198185: 4, 198191: 6}",Consumable,uncommon,300.0,327.0,15.00,...,16.8,36.2,100.0,100.0,16.8,16.8,16.8,16.8,100.0,100.0
422,Engineering,None,198212,EZ-Thro Primal Deconstruction Charge,"{190320: 1, 198191: 4, 198188: 1, 198185: 4, 1...",Consumable,uncommon,250.0,327.0,12.50,...,100.0,100.0,100.0,100.0,16.8,16.8,100.0,100.0,100.0,100.0


In [5]:
list(all_data.columns)

['profession',
 'character',
 'id',
 'item',
 'reagents',
 'tag',
 'rarity',
 'difficulty',
 'skill1',
 'hsv',
 'inspiration_skill',
 'inspiration_percent',
 'base_quantity',
 'multicraft_percent',
 'has_reagent_qualities',
 'has_embellishment_slot',
 'has_missive_slot',
 'has_tinker_slot',
 'has_crest_slot',
 'icon',
 'skill2',
 'skill3',
 'difficulty_tinker',
 'difficulty_missive',
 'difficulty_embellishment',
 'difficulty_tinker_missive',
 'difficulty_missive_embellishment',
 'difficulty_whelpling',
 'difficulty_whelpling_tinker',
 'difficulty_whelpling_missive',
 'difficulty_whelpling_embellishment',
 'difficulty_whelpling_tinker_missive',
 'difficulty_whelpling_missive_embellishment',
 'difficulty_wyrm',
 'difficulty_wyrm_tinker',
 'difficulty_wyrm_missive',
 'difficulty_wyrm_embellishment',
 'difficulty_wyrm_tinker_missive',
 'difficulty_wyrm_missive_embellishment',
 'difficulty_aspect',
 'difficulty_aspect_tinker',
 'difficulty_aspect_missive',
 'difficulty_aspect_embellishment'

In [6]:
all_data

,profession,character,id,item,reagents,tag,rarity,difficulty,skill1,hsv,...,odds_aspect_embellishment2,odds_aspect_embellishment_hsv2,odds_aspect_embellishment3,odds_aspect_embellishment_hsv3,odds_aspect_missive_embellishment1,odds_aspect_missive_embellishment_hsv1,odds_aspect_missive_embellishment2,odds_aspect_missive_embellishment_hsv2,odds_aspect_missive_embellishment3,odds_aspect_missive_embellishment_hsv3
8,Alchemy,Trillithia,191386,Aerated Mana Potion,"{190326: 3, 191476: 5, 191462: 8}",Consumable,common,60.0,272.0,3.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
29,Alchemy,Trillithia,191344,Aerated Phial of Deftness,"{190456: 10, 190327: 2, 191476: 10, 191466: 10}",Consumable,common,400.0,272.0,20.00,...,36.8,36.8,36.8,36.8,36.8,36.8,36.8,36.8,36.8,36.8
31,Alchemy,Trillithia,197722,Aerated Phial of Quick Hands,"{190326: 4, 191476: 2, 191462: 8}",Consumable,common,200.0,272.0,10.00,...,100.0,100.0,100.0,100.0,21.8,21.8,100.0,100.0,100.0,100.0
49,Alchemy,Trillithia,191522,Agitating Potion Augmentation,"{191466: 5, 191462: 12, 191469: 6}",Crafting Reagent,uncommon,300.0,222.0,15.00,...,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8
56,Alchemy,Trillithia,191492,Alacritous Alchemist Stone,"{206959: 1, 201406: 1, 191498: 15, 191389: 12,...",Gear (Combat),epic,275.0,232.0,13.75,...,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8,34.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,Tailoring,Linidel,193539,Wildercloth Enchanter's Hat,"{193922: 3, 194123: 3, 192097: 4}",Gear (Profession),uncommon,80.0,276.0,4.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
889,Tailoring,Linidel,193529,Wildercloth Fishing Cap,"{193922: 5, 192097: 4}",Gear (Profession),uncommon,80.0,276.0,4.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
891,Tailoring,Linidel,193538,Wildercloth Gardening Hat,"{193922: 5, 192097: 4, 191462: 5}",Gear (Profession),uncommon,80.0,276.0,4.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
892,Tailoring,Linidel,193541,Wildercloth Tailor's Coat,"{193922: 6, 192097: 4}",Gear (Profession),uncommon,80.0,276.0,4.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
